In [2]:
import os, random, time
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as TF
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

from tqdm import tqdm
from torch.amp import autocast
from torch.cuda.amp import autocast, GradScaler

# --- 경로 설정 및 하이퍼파라미터 ---
BASE_DIR     = f"./data/covid19-xray-severity-scoring/"
CSV_PATH     = str(Path(BASE_DIR) / "Brixia.csv")
IMAGE_DIR    = str(Path(BASE_DIR) / "segmented_png")

OUT_DIR      = "./runs_severity_classification"
BEST_PATH    = str(Path(OUT_DIR) / "best_efficientnet_b0_classification.pth")
PHASE2_PATH  = str(Path(OUT_DIR) / "phase2_weighted_classification.pth")
os.makedirs(OUT_DIR, exist_ok=True)

DEVICE       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED         = 42
IMG_SIZE     = 224
BATCH_SIZE   = 128  # 🔄 분류는 배치 크기 줄임
NUM_CLASSES  = 4   # ✅ NEW: 0, 1, 2, 3
EPOCHS_PHASE1 = 30
EPOCHS_PHASE2 = 50
LR           = 1e-4
WEIGHT_DECAY = 5e-4
AMP          = True
EARLY_STOP_ACC = 0.75  # 🔄 MAE → Accuracy
DROP_RATIO   = 0.3
AUG_RATIO    = 0.5
MIXUP_ALPHA  = 0.2
LABEL_SMOOTHING = 0.1  # ✅ NEW: Label smoothing

# --- 시드 고정 ---
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
set_seed(SEED)

def make_transform_with_label(train: bool, img_size: int = IMG_SIZE, aug_ratio=AUG_RATIO):
    """Brixia Score의 좌우 구조를 고려한 transform"""
    def _tfm(img: Image.Image, label: torch.Tensor = None):
        img = img.convert('RGB')
        img = TF.resize(
            img, 
            [img_size, img_size], 
            interpolation=TF.InterpolationMode.BILINEAR,
            antialias=True
        )
        
        if train:
            # 1. Horizontal Flip (라벨도 함께 flip)
            if random.random() < aug_ratio:
                img = TF.hflip(img)
                if label is not None:
                    label = label[[3, 4, 5, 0, 1, 2]]
            
            # 2. 약한 회전 (±5도)
            if random.random() < aug_ratio:
                angle = float(torch.empty(1).uniform_(-5, 5))
                img = TF.rotate(
                    img, 
                    angle, 
                    interpolation=TF.InterpolationMode.BILINEAR,
                    fill=0
                )
            
            # 3. 약한 Translation
            if random.random() < aug_ratio:
                max_dx = 0.05 * img_size
                max_dy = 0.05 * img_size
                translations = (
                    float(torch.empty(1).uniform_(-max_dx, max_dx)),
                    float(torch.empty(1).uniform_(-max_dy, max_dy))
                )
                img = TF.affine(
                    img,
                    angle=0,
                    translate=translations,
                    scale=1.0,
                    shear=0,
                    interpolation=TF.InterpolationMode.BILINEAR,
                    fill=0
                )
            
            # 4. Brightness & Contrast
            if random.random() < aug_ratio:
                brightness_factor = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_brightness(img, brightness_factor)
            
            if random.random() < aug_ratio:
                contrast_factor = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_contrast(img, contrast_factor)
            
            # 5. Gamma Correction
            if random.random() < 0.3:
                gamma = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_gamma(img, gamma)
        
        # Tensor 변환
        img = TF.to_tensor(img)
        
        # Gaussian Noise (train only)
        if train and random.random() < 0.2:
            noise = torch.randn_like(img) * 0.01
            img = img + noise
            img = torch.clamp(img, 0, 1)
        
        # 정규화
        img = TF.normalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        if label is not None:
            return img, label
        return img
    
    return _tfm

def load_and_split_brixia(csv_path, val_ratio=0.2, seed=SEED):
    df = pd.read_csv(csv_path, dtype={'BrixiaScore': str})
    df = df.dropna(subset=['BrixiaScore'])
    df = df[df['BrixiaScore'] != 'nan']
    df = df[df['BrixiaScore'].str.len() == 6].copy()
    
    print(f"유효한 데이터: {len(df)}개")
    
    if 'ConsensusTestset' in df.columns:
        test_df = df[df['ConsensusTestset'] == 1].copy()
        train_val_df = df[df['ConsensusTestset'] == 0].copy()
    else:
        test_df = pd.DataFrame()
        train_val_df = df.copy()
    
    gss = GroupShuffleSplit(n_splits=1, test_size=val_ratio, random_state=seed)
    train_idx, val_idx = next(gss.split(
        train_val_df, 
        groups=train_val_df['StudyId']
    ))
    
    tr_df = train_val_df.iloc[train_idx].copy()
    val_df = train_val_df.iloc[val_idx].copy()
    
    print(f"Train: {len(tr_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
    validate_split(tr_df, val_df, test_df)
    
    return tr_df, val_df, test_df

def validate_split(tr_df, val_df, tt_df):
    train_studies = set(tr_df['StudyId'])
    val_studies = set(val_df['StudyId'])
    test_studies = set(tt_df['StudyId']) if len(tt_df) > 0 else set()
    
    assert len(train_studies & val_studies) == 0, "Train-Val 간 StudyId 중복!"
    assert len(train_studies & test_studies) == 0, "Train-Test 간 StudyId 중복!"
    assert len(val_studies & test_studies) == 0, "Val-Test 간 StudyId 중복!"
    
    for name, data in [('Train', tr_df), ('Val', val_df), ('Test', tt_df)]:
        if len(data) > 0:
            scores = data['BrixiaScore'].apply(lambda x: sum(int(c) for c in x))
            print(f"{name} - Mean: {scores.mean():.2f}, Std: {scores.std():.2f}")
    
    return True

# ============================================================
# Dataset
# ============================================================
class BrixiaDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.img_col = "Filename"
        self.label_col = "BrixiaScore"
        self._validate_data()
    
    def _validate_data(self):
        assert self.img_col in self.df.columns
        assert self.label_col in self.df.columns
        
        invalid_scores = self.df[self.df[self.label_col].str.len() != 6]
        if len(invalid_scores) > 0:
            print(f"⚠️ 경고: {len(invalid_scores)}개의 잘못된 BrixiaScore 발견")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        img_name_from_csv = row[self.img_col]
        img_name = img_name_from_csv.replace('.dcm', '.png')
        img_path = os.path.join(self.img_dir, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"❌ 이미지 로드 오류: {img_path}")
            raise
        
        scores_str = row[self.label_col]
        scores_list = [int(c) for c in scores_str]
        labels = torch.tensor(scores_list, dtype=torch.long)  # 🔄 long으로 변경
        
        if self.transform:
            image, labels = self.transform(image, labels)
        else:
            image = TF.to_tensor(image)
            image = TF.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, labels

def create_dataloaders(tr_df, val_df, tt_df, img_dir, 
                       batch_size=32, img_size=224, num_workers=4):
    train_transform = make_transform_with_label(train=True, img_size=img_size)
    val_transform = make_transform_with_label(train=False, img_size=img_size)
    
    tr_ds = BrixiaDataset(tr_df, img_dir, transform=train_transform)
    val_ds = BrixiaDataset(val_df, img_dir, transform=val_transform)
    tt_ds = BrixiaDataset(tt_df, img_dir, transform=val_transform)
    
    tr_loader = DataLoader(
        tr_ds, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available(),
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_ds, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )
    
    tt_loader = DataLoader(
        tt_ds, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )
    
    print("✅ DataLoader 준비 완료")
    print(f"   Train: {len(tr_ds)} samples, {len(tr_loader)} batches")
    print(f"   Val:   {len(val_ds)} samples, {len(val_loader)} batches")
    print(f"   Test:  {len(tt_ds)} samples, {len(tt_loader)} batches")
    
    return tr_loader, val_loader, tt_loader

# ============================================================
# Loss Function (분류용으로 변경)
# ============================================================
def calculate_class_weights(labels, num_classes=4, method='sqrt_inverse'):
    """클래스 불균형 해결을 위한 가중치 계산"""
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    labels_flat = labels.flatten()
    counts = np.bincount(labels_flat.astype(int), minlength=num_classes)
    
    if method == 'sqrt_inverse':
        weights = 1.0 / (np.sqrt(counts) + 1e-6)
    elif method == 'inverse':
        weights = 1.0 / (counts + 1e-6)
    else:
        total = len(labels_flat)
        weights = total / (num_classes * (counts + 1e-6))
    
    weights = weights / weights.mean()
    return torch.FloatTensor(weights)

def print_class_distribution(labels):
    """클래스 분포 시각화"""
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    print("=" * 60)
    print("Class Distribution Analysis")
    print("=" * 60)
    
    region_names = ['A', 'B', 'C', 'D', 'E', 'F']
    for idx, name in enumerate(region_names):
        region_labels = labels[:, idx]
        counts = np.bincount(region_labels.astype(int), minlength=4)
        total = counts.sum()
        
        print(f"\n{name}:")
        for cls in range(4):
            pct = 100 * counts[cls] / total if total > 0 else 0
            bar = '█' * int(pct / 2)
            print(f"  Class {cls}: {counts[cls]:4d} ({pct:5.1f}%) {bar}")
    
    print("=" * 60)

class AdaptiveClassificationLoss(nn.Module):
    """
    ✅ NEW: 다중 클래스 분류를 위한 손실함수
    - CrossEntropyLoss 기반
    - 클래스 가중치 적용
    - 부위별 가중치 적용
    - Label Smoothing 지원
    """
    
    def __init__(self, train_labels, num_classes=4, use_class_weights=True, 
                 part_weights=None, label_smoothing=0.1):
        super().__init__()
        self.num_classes = num_classes
        self.label_smoothing = label_smoothing
        
        if isinstance(train_labels, torch.Tensor):
            train_labels_np = train_labels.cpu().numpy()
        else:
            train_labels_np = train_labels
        
        print_class_distribution(train_labels_np)
        
        # 클래스 가중치
        if use_class_weights:
            class_weights = calculate_class_weights(train_labels_np, num_classes=num_classes)
            self.register_buffer('class_weights', class_weights)
            print(f"✅ Class weights: {class_weights.numpy()}")
        else:
            self.class_weights = None
        
        # 부위별 가중치
        if part_weights is None:
            self.part_weights = torch.ones(6)
        else:
            self.part_weights = torch.tensor(part_weights, dtype=torch.float32)
        self.register_buffer('part_weights_buf', self.part_weights)
        print(f"✅ Part weights: {self.part_weights.numpy()}")

    def forward(self, pred_logits, target, use_mixup=False):
        """
        pred_logits: [B, 6, 4] - 각 부위별 4개 클래스에 대한 logits
        target: [B, 6] - 각 부위별 클래스 레이블 (0~3)
        """
        B, num_regions, num_classes = pred_logits.shape
        
        # Reshape for loss computation
        pred_logits_flat = pred_logits.view(B * num_regions, num_classes)  # [B*6, 4]
        target_flat = target.view(B * num_regions)  # [B*6]
        
        # CrossEntropyLoss with label smoothing
        if self.class_weights is not None and not use_mixup:
            criterion = nn.CrossEntropyLoss(
                weight=self.class_weights.to(pred_logits.device),
                label_smoothing=self.label_smoothing,
                reduction='none'
            )
        else:
            criterion = nn.CrossEntropyLoss(
                label_smoothing=self.label_smoothing,
                reduction='none'
            )
        
        loss = criterion(pred_logits_flat, target_flat)  # [B*6]
        loss = loss.view(B, num_regions)  # [B, 6]
        
        # 부위별 가중치 적용
        part_weights = self.part_weights_buf.to(pred_logits.device)
        loss = loss * part_weights.unsqueeze(0)  # [B, 6]
        
        # 부위별 평균 손실 (모니터링용)
        part_losses = loss.mean(dim=0)  # [6]
        
        return loss.mean(), part_losses

# ============================================================
# Mixup (분류용)
# ============================================================
def mixup_data_classification(x, y, alpha=MIXUP_ALPHA):
    """
    ✅ NEW: 분류용 Mixup
    y는 one-hot으로 변환 후 mixup
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    
    # One-hot encoding
    y_onehot = F.one_hot(y, num_classes=NUM_CLASSES).float()  # [B, 6, 4]
    y_onehot_shuffled = y_onehot[index]
    
    mixed_y = lam * y_onehot + (1 - lam) * y_onehot_shuffled  # [B, 6, 4]
    
    return mixed_x, mixed_y, lam

def mixup_criterion(criterion, pred_logits, y_mixed, lam):
    """Mixup을 위한 손실 계산"""
    # y_mixed는 [B, 6, 4] soft labels
    # pred_logits는 [B, 6, 4]
    
    B, num_regions, num_classes = pred_logits.shape
    
    # Reshape
    pred_flat = pred_logits.view(B * num_regions, num_classes)  # [B*6, 4]
    target_flat = y_mixed.view(B * num_regions, num_classes)  # [B*6, 4]
    
    # Soft target loss
    log_probs = F.log_softmax(pred_flat, dim=1)
    loss = -(target_flat * log_probs).sum(dim=1)  # [B*6]
    loss = loss.view(B, num_regions)  # [B, 6]
    
    # 부위별 가중치 적용 (criterion에서 가져옴)
    if hasattr(criterion, 'part_weights_buf'):
        part_weights = criterion.part_weights_buf.to(pred_logits.device)
        loss = loss * part_weights.unsqueeze(0)
    
    part_losses = loss.mean(dim=0)
    return loss.mean(), part_losses

# ============================================================
# Metrics (분류용)
# ============================================================
@torch.no_grad()
def calculate_classification_metrics(pred_logits, labels):
    """
    ✅ NEW: 분류 지표 계산
    pred_logits: [B, 6, 4]
    labels: [B, 6]
    """
    # 예측 클래스
    preds = pred_logits.argmax(dim=-1)  # [B, 6]
    
    # Exact match accuracy
    exact_acc = (preds == labels).float().mean().item()
    
    # Off-by-1 accuracy (인접 클래스 허용)
    off_by_1 = (torch.abs(preds - labels) <= 1).float().mean().item()
    
    # Per-region accuracy
    region_acc = (preds == labels).float().mean(dim=0)  # [6]
    
    # MAE (참고용)
    mae = torch.abs(preds.float() - labels.float()).mean().item()
    
    return exact_acc, off_by_1, mae, region_acc

# ============================================================
# Model (분류용으로 변경)
# ============================================================
class EfficientNetB0Classification(nn.Module):
    """
    ✅ NEW: 분류를 위한 모델
    각 부위(A~F)마다 4개 클래스(0~3) 예측
    """
    def __init__(self, pretrained=True, drop=0.3, num_regions=6, num_classes=4):
        super().__init__()
        
        weights = EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None
        backbone = efficientnet_b0(weights=weights)
        
        self.features = backbone.features
        in_feat = 1280
        
        # Positional embedding
        self.pos_embed = nn.Parameter(torch.randn(1, 49, in_feat))
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        
        # Region queries
        self.region_queries = nn.Parameter(torch.randn(num_regions, in_feat))
        nn.init.xavier_uniform_(self.region_queries)
        
        # Cross attention
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=in_feat,
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )
        
        self.norm1 = nn.LayerNorm(in_feat)
        self.norm2 = nn.LayerNorm(in_feat)
        
        # FFN
        self.ffn = nn.Sequential(
            nn.Linear(in_feat, in_feat * 2),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(in_feat * 2, in_feat),
            nn.Dropout(drop)
        )
        
        # 🔄 Classification heads: 각 부위마다 4개 클래스 예측
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(in_feat, 256),
                nn.LayerNorm(256),
                nn.GELU(),
                nn.Dropout(drop),
                nn.Linear(256, 128),
                nn.LayerNorm(128),
                nn.GELU(),
                nn.Dropout(drop),
                nn.Linear(128, num_classes)  # 🔄 4개 클래스 출력
            ) for _ in range(num_regions)
        ])
    
    def forward(self, x):
        B = x.size(0)
        
        # Feature extraction
        feat = self.features(x)  # [B, 1280, 7, 7]
        feat = feat.flatten(2).transpose(1, 2)  # [B, 49, 1280]
        feat = feat + self.pos_embed
        
        # Region queries
        queries = self.region_queries.unsqueeze(0).expand(B, -1, -1)  # [B, 6, 1280]
        
        # Cross attention
        attn_out, _ = self.cross_attention(
            query=queries,
            key=feat,
            value=feat
        )
        
        attn_out = self.norm1(attn_out + queries)
        ffn_out = self.ffn(attn_out)
        attn_out = self.norm2(attn_out + ffn_out)  # [B, 6, 1280]
        
        # 🔄 각 부위별 classification head 통과
        outputs = []
        for i in range(len(self.heads)):
            region_feat = attn_out[:, i, :]  # [B, 1280]
            logits = self.heads[i](region_feat)  # [B, 4]
            outputs.append(logits)
        
        out = torch.stack(outputs, dim=1)  # [B, 6, 4]
        return out

# ============================================================
# Training Functions (분류용 수정)
# ============================================================
def train_epoch(model, tr_loader, criterion, optimizer, scaler, device, 
                amp=True, use_mixup=True):
    model.train()
    run_loss = run_acc = run_off1 = run_mae = n = 0
    part_losses_sum = torch.zeros(6)
    
    pbar = tqdm(tr_loader, desc="Train", leave=False)
    
    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)  # [B, 6]
        
        # Mixup 적용
        is_mixup = use_mixup and (random.random() < 0.5)
        
        optimizer.zero_grad(set_to_none=True)
        
        with autocast(enabled=amp):
            pred_logits = model(imgs)  # [B, 6, 4]
            
            if is_mixup:
                imgs_mixed, labels_mixed, lam = mixup_data_classification(imgs, labels)
                pred_logits = model(imgs_mixed)
                loss, part_losses = mixup_criterion(criterion, pred_logits, labels_mixed, lam)
            else:
                loss, part_losses = criterion(pred_logits, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Metrics (원본 labels로 계산)
        exact_acc, off_by_1, mae, _ = calculate_classification_metrics(pred_logits.detach(), labels)
        
        bs = imgs.size(0)
        run_loss += loss.item() * bs
        run_acc += exact_acc * bs
        run_off1 += off_by_1 * bs
        run_mae += mae * bs
        part_losses_sum += part_losses.cpu() * bs
        n += bs
        
        pbar.set_postfix(
            loss=f"{run_loss/n:.4f}",
            acc=f"{run_acc/n:.4f}",
            mae=f"{run_mae/n:.4f}"
        )
    
    part_losses_avg = part_losses_sum / n
    
    return run_loss/n, run_acc/n, run_off1/n, run_mae/n, part_losses_avg

@torch.no_grad()
def evaluate(model, val_loader, criterion, device, split='val'):
    model.eval()
    run_loss = run_acc = run_off1 = run_mae = n = 0
    part_losses_sum = torch.zeros(6)
    region_acc_sum = torch.zeros(6)
    
    pbar = tqdm(val_loader, desc=f"{split.capitalize()}", leave=False)
    
    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        pred_logits = model(imgs)  # [B, 6, 4]
        loss, part_losses = criterion(pred_logits, labels)
        
        exact_acc, off_by_1, mae, region_acc = calculate_classification_metrics(pred_logits, labels)
        
        bs = imgs.size(0)
        run_loss += loss.item() * bs
        run_acc += exact_acc * bs
        run_off1 += off_by_1 * bs
        run_mae += mae * bs
        part_losses_sum += part_losses.cpu() * bs
        region_acc_sum += region_acc.cpu() * bs
        n += bs
        
        pbar.set_postfix(
            loss=f"{run_loss/n:.4f}",
            acc=f"{run_acc/n:.4f}"
        )
    
    avg_loss = run_loss/n
    avg_acc = run_acc/n
    avg_off1 = run_off1/n
    avg_mae = run_mae/n
    part_losses_avg = part_losses_sum / n
    region_acc_avg = region_acc_sum / n
    
    print(f"[{split}] loss:{avg_loss:.4f} acc:{avg_acc:.4f} "
          f"off1:{avg_off1:.4f} mae:{avg_mae:.4f}")
    print(f"  Region Acc: {region_acc_avg.numpy().round(3)}")
    
    return avg_loss, avg_acc, avg_off1, avg_mae, part_losses_avg, region_acc_avg

def get_lrs(optimizer):
    return [pg['lr'] for pg in optimizer.param_groups]

# ============================================================
# Main Function
# ============================================================
def main():
    print("\n" + "="*70)
    print("🚀 Two-Phase Classification Training")
    print("="*70)
    
    # 데이터 준비
    print("\n📂 Loading data...")
    tr_df, val_df, tt_df = load_and_split_brixia(CSV_PATH)
    
    print("\n📦 Creating DataLoaders...")
    tr_loader, val_loader, tt_loader = create_dataloaders(
        tr_df, val_df, tt_df, img_dir=IMAGE_DIR, 
        batch_size=BATCH_SIZE, img_size=IMG_SIZE, num_workers=4
    )
    
    # Train labels 추출
    train_labels = torch.cat([labels for _, labels in tr_loader], dim=0)
    
    # ========================================
    # Phase 1: 균등 가중치 학습
    # ========================================
    print("\n" + "="*70)
    print("📍 PHASE 1: Training with Uniform Weights")
    print("="*70)
    
    criterion_phase1 = AdaptiveClassificationLoss(
        train_labels, 
        num_classes=NUM_CLASSES,
        use_class_weights=True,
        part_weights=None,
        label_smoothing=LABEL_SMOOTHING
    )
    
    model = EfficientNetB0Classification(
        pretrained=True, 
        drop=DROP_RATIO,
        num_regions=6,
        num_classes=NUM_CLASSES
    ).to(DEVICE)
    print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, min_lr=1e-6
    )
    scaler = GradScaler(enabled=AMP)
    
    best_acc_phase1 = 0.0
    patience_counter = 0
    max_patience = 10
    
    # Phase 1 학습
    phase1_part_losses = []
    
    for ep in range(1, EPOCHS_PHASE1 + 1):
        t0 = time.time()
        
        tr_loss, tr_acc, tr_off1, tr_mae, tr_part_losses = train_epoch(
            model, tr_loader, criterion_phase1, optimizer, scaler, DEVICE, AMP
        )
        
        val_loss, val_acc, val_off1, val_mae, val_part_losses, val_region_acc = evaluate(
            model, val_loader, criterion_phase1, DEVICE, split='val'
        )
        
        scheduler.step(val_acc)
        
        # 부위별 손실 누적
        phase1_part_losses.append(val_part_losses.numpy())
        
        # Save best
        if val_acc > best_acc_phase1:
            best_acc_phase1 = val_acc
            patience_counter = 0
            torch.save({
                'epoch': ep,
                'model_state_dict': model.state_dict(),
                'val_acc': best_acc_phase1,
                'val_mae': val_mae,
            }, BEST_PATH)
            print(f"✅ Phase 1 Best (Acc={best_acc_phase1:.4f}, MAE={val_mae:.4f})")
        else:
            patience_counter += 1
        
        if patience_counter >= max_patience:
            print(f"\n⏹️ Phase 1 Early stopping at epoch {ep}")
            break
        
        elapsed = time.time() - t0
        print(f"\n[Phase1 Epoch {ep:02d}/{EPOCHS_PHASE1}]")
        print(f"  Train - acc:{tr_acc:.4f} off1:{tr_off1:.4f} mae:{tr_mae:.4f}")
        print(f"  Val   - acc:{val_acc:.4f} off1:{val_off1:.4f} mae:{val_mae:.4f}")
        print(f"  Part losses (Val): {val_part_losses.numpy().round(3)}")
        print(f"  LR:{get_lrs(optimizer)[0]:.2e} | {elapsed:.1f}s | Pat:{patience_counter}/{max_patience}")
        print("-" * 70)
    
    # Phase 1 부위별 평균 손실 계산
    avg_part_losses = np.mean(phase1_part_losses, axis=0)
    print("\n" + "="*70)
    print("📊 Phase 1 Average Part Losses:")
    region_names = ['A', 'B', 'C', 'D', 'E', 'F']
    for i, name in enumerate(region_names):
        print(f"   {name}: {avg_part_losses[i]:.4f}")
    print("="*70)
    
    # ========================================
    # Phase 2: 난이도 기반 가중치
    # ========================================
    print("\n" + "="*70)
    print("📍 PHASE 2: Training with Difficulty-Based Weights")
    print("="*70)
    
    # 난이도 기반 가중치 계산
    normalized_losses = avg_part_losses / avg_part_losses.mean()
    part_weights_phase2 = normalized_losses ** 0.5
    part_weights_phase2 = part_weights_phase2.tolist()
    
    print(f"\n🎯 Calculated Part Weights:")
    for i, name in enumerate(region_names):
        print(f"   {name}: {part_weights_phase2[i]:.3f} (loss: {avg_part_losses[i]:.4f})")
    
    # Phase 2 criterion
    criterion_phase2 = AdaptiveClassificationLoss(
        train_labels,
        num_classes=NUM_CLASSES,
        use_class_weights=True,
        part_weights=part_weights_phase2,
        label_smoothing=LABEL_SMOOTHING
    )
    
    # Phase 1 best model 로드
    model.load_state_dict(torch.load(BEST_PATH)['model_state_dict'])
    
    # 새로운 optimizer & scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR*0.5, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, min_lr=1e-7
    )
    scaler = GradScaler(enabled=AMP)
    
    best_acc_phase2 = 0.0
    patience_counter = 0
    
    # Phase 2 학습
    for ep in range(1, EPOCHS_PHASE2 + 1):
        t0 = time.time()
        
        tr_loss, tr_acc, tr_off1, tr_mae, tr_part_losses = train_epoch(
            model, tr_loader, criterion_phase2, optimizer, scaler, DEVICE, AMP
        )
        
        val_loss, val_acc, val_off1, val_mae, val_part_losses, val_region_acc = evaluate(
            model, val_loader, criterion_phase2, DEVICE, split='val'
        )
        
        scheduler.step(val_acc)
        
        # Early stopping check
        if val_acc >= EARLY_STOP_ACC:
            print(f"\n✅ Target Acc {EARLY_STOP_ACC} reached at epoch {ep}!")
            torch.save({
                'epoch': ep,
                'model_state_dict': model.state_dict(),
                'val_acc': val_acc,
                'val_mae': val_mae,
                'part_weights': part_weights_phase2
            }, PHASE2_PATH)
            break
        
        # Save best
        if val_acc > best_acc_phase2:
            best_acc_phase2 = val_acc
            patience_counter = 0
            torch.save({
                'epoch': ep,
                'model_state_dict': model.state_dict(),
                'val_acc': best_acc_phase2,
                'val_mae': val_mae,
                'part_weights': part_weights_phase2
            }, PHASE2_PATH)
            print(f"✅ Phase 2 Best (Acc={best_acc_phase2:.4f}, MAE={val_mae:.4f})")
        else:
            patience_counter += 1
        
        if patience_counter >= max_patience:
            print(f"\n⏹️ Phase 2 Early stopping at epoch {ep}")
            break
        
        elapsed = time.time() - t0
        print(f"\n[Phase2 Epoch {ep:02d}/{EPOCHS_PHASE2}]")
        print(f"  Train - acc:{tr_acc:.4f} off1:{tr_off1:.4f} mae:{tr_mae:.4f}")
        print(f"  Val   - acc:{val_acc:.4f} off1:{val_off1:.4f} mae:{val_mae:.4f}")
        print(f"  Part losses (Val): {val_part_losses.numpy().round(3)}")
        print(f"  LR:{get_lrs(optimizer)[0]:.2e} | {elapsed:.1f}s | Pat:{patience_counter}/{max_patience}")
        print("-" * 70)
    
    # ========================================
    # Test Evaluation
    # ========================================
    print("\n" + "="*70)
    print("🎉 Training Finished!")
    print("="*70)
    
    if len(tt_loader) > 0:
        print("\n📊 Test evaluation with Phase 2 model...")
        model.load_state_dict(torch.load(PHASE2_PATH)['model_state_dict'])
        tt_loss, tt_acc, tt_off1, tt_mae, tt_part_losses, tt_region_acc = evaluate(
            model, tt_loader, criterion_phase2, DEVICE, split='test'
        )
        print(f"\n🏆 Test Results:")
        print(f"   Accuracy: {tt_acc:.4f}")
        print(f"   Off-by-1: {tt_off1:.4f}")
        print(f"   MAE: {tt_mae:.4f}")
        print(f"   Region Acc: {tt_region_acc.numpy().round(3)}")
        print(f"   Part losses: {tt_part_losses.numpy().round(3)}")
    
    print(f"\n💾 Phase 1 model saved: {BEST_PATH}")
    print(f"💾 Phase 2 model saved: {PHASE2_PATH}")
    
    print("\n📈 Summary:")
    print(f"   Phase 1 Best Acc: {best_acc_phase1:.4f}")
    print(f"   Phase 2 Best Acc: {best_acc_phase2:.4f}")
    print(f"   Improvement: {(best_acc_phase2 - best_acc_phase1):.4f}")

if __name__ == "__main__":
    main()


🚀 Two-Phase Classification Training

📂 Loading data...
유효한 데이터: 4695개
Train: 3637, Val: 912, Test: 146
Train - Mean: 8.31, Std: 4.26
Val - Mean: 8.35, Std: 4.15
Test - Mean: 7.78, Std: 4.20

📦 Creating DataLoaders...
✅ DataLoader 준비 완료
   Train: 3637 samples, 28 batches
   Val:   912 samples, 8 batches
   Test:  146 samples, 2 batches

📍 PHASE 1: Training with Uniform Weights
Class Distribution Analysis

A:
  Class 0: 1791 ( 50.0%) ████████████████████████
  Class 1: 1122 ( 31.3%) ███████████████
  Class 2:  457 ( 12.8%) ██████
  Class 3:  214 (  6.0%) ██

B:
  Class 0:  740 ( 20.6%) ██████████
  Class 1:  912 ( 25.4%) ████████████
  Class 2: 1159 ( 32.3%) ████████████████
  Class 3:  773 ( 21.6%) ██████████

C:
  Class 0:  392 ( 10.9%) █████
  Class 1:  797 ( 22.2%) ███████████
  Class 2: 1295 ( 36.1%) ██████████████████
  Class 3: 1100 ( 30.7%) ███████████████

D:
  Class 0: 1791 ( 50.0%) ████████████████████████
  Class 1: 1114 ( 31.1%) ███████████████
  Class 2:  452 ( 12.6%) ████

[val] loss:1.2036 acc:0.4492 off1:0.8598 mae:0.7092
  Region Acc: [0.49  0.394 0.404 0.593 0.404 0.411]
✅ Phase 1 Best (Acc=0.4492, MAE=0.7092)

[Phase1 Epoch 01/30]
  Train - acc:0.3878 off1:0.8030 mae:0.8513
  Val   - acc:0.4492 off1:0.8598 mae:0.7092
  Part losses (Val): [1.228 1.262 1.242 1.015 1.245 1.231]
  LR:1.00e-04 | 18.5s | Pat:0/10
----------------------------------------------------------------------


[val] loss:1.1401 acc:0.4960 off1:0.9077 mae:0.6043
  Region Acc: [0.537 0.461 0.458 0.603 0.459 0.457]
✅ Phase 1 Best (Acc=0.4960, MAE=0.6043)

[Phase1 Epoch 02/30]
  Train - acc:0.4324 off1:0.8436 mae:0.7551
  Val   - acc:0.4960 off1:0.9077 mae:0.6043
  Part losses (Val): [1.134 1.195 1.191 0.991 1.15  1.182]
  LR:1.00e-04 | 13.1s | Pat:0/10
----------------------------------------------------------------------


[val] loss:1.1196 acc:0.5217 off1:0.9092 mae:0.5740
  Region Acc: [0.579 0.473 0.481 0.638 0.495 0.465]
✅ Phase 1 Best (Acc=0.5217, MAE=0.5740)

[Phase1 Epoch 03/30]
  Train - acc:0.4648 off1:0.8708 mae:0.6828
  Val   - acc:0.5217 off1:0.9092 mae:0.5740
  Part losses (Val): [1.111 1.165 1.172 0.969 1.135 1.166]
  LR:1.00e-04 | 13.5s | Pat:0/10
----------------------------------------------------------------------


[val] loss:1.1032 acc:0.5256 off1:0.9174 mae:0.5610
  Region Acc: [0.559 0.496 0.486 0.598 0.518 0.498]
✅ Phase 1 Best (Acc=0.5256, MAE=0.5610)

[Phase1 Epoch 04/30]
  Train - acc:0.4704 off1:0.8710 mae:0.6797
  Val   - acc:0.5256 off1:0.9174 mae:0.5610
  Part losses (Val): [1.102 1.132 1.162 0.964 1.103 1.156]
  LR:1.00e-04 | 14.1s | Pat:0/10
----------------------------------------------------------------------


...

[val] loss:1.0649 acc:0.5806 off1:0.9538 mae:0.4693
  Region Acc: [0.589 0.569 0.561 0.652 0.588 0.524]

[Phase2 Epoch 08/50]
  Train - acc:0.5572 off1:0.9007 mae:0.5642
  Val   - acc:0.5806 off1:0.9538 mae:0.4693
  Part losses (Val): [1.043 1.085 1.118 0.882 1.062 1.199]
  LR:2.50e-05 | 12.7s | Pat:7/10
----------------------------------------------------------------------


[val] loss:1.0759 acc:0.5821 off1:0.9545 mae:0.4667
  Region Acc: [0.601 0.561 0.565 0.651 0.587 0.527]

[Phase2 Epoch 09/50]
  Train - acc:0.5859 off1:0.9128 mae:0.5196
  Val   - acc:0.5821 off1:0.9545 mae:0.4667
  Part losses (Val): [1.056 1.096 1.127 0.888 1.078 1.21 ]
  LR:2.50e-05 | 12.9s | Pat:8/10
----------------------------------------------------------------------


[val] loss:1.0793 acc:0.5777 off1:0.9525 mae:0.4733
  Region Acc: [0.598 0.567 0.561 0.649 0.58  0.511]

[Phase2 Epoch 10/50]
  Train - acc:0.6305 off1:0.9409 mae:0.4383
  Val   - acc:0.5777 off1:0.9525 mae:0.4733
  Part losses (Val): [1.062 1.096 1.131 0.895 1.076 1.216]
  LR:2.50e-05 | 13.2s | Pat:9/10
----------------------------------------------------------------------


[val] loss:1.0792 acc:0.5760 off1:0.9518 mae:0.4762
  Region Acc: [0.603 0.557 0.56  0.654 0.566 0.516]

⏹️ Phase 2 Early stopping at epoch 11

🎉 Training Finished!

📊 Test evaluation with Phase 2 model...


[test] loss:1.0002 acc:0.5936 off1:0.9658 mae:0.4418
  Region Acc: [0.623 0.589 0.521 0.726 0.568 0.534]

🏆 Test Results:
   Accuracy: 0.5936
   Off-by-1: 0.9658
   MAE: 0.4418
   Region Acc: [0.623 0.589 0.521 0.726 0.568 0.534]
   Part losses: [0.91  1.081 1.075 0.787 0.997 1.15 ]

💾 Phase 1 model saved: runs_severity_classification/best_efficientnet_b0_classification.pth
💾 Phase 2 model saved: runs_severity_classification/phase2_weighted_classification.pth

📈 Summary:
   Phase 1 Best Acc: 0.5853
   Phase 2 Best Acc: 0.5837
   Improvement: -0.0016


In [3]:
import os, random, time
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as TF
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

from tqdm import tqdm
from torch.amp import autocast
from torch.cuda.amp import autocast, GradScaler

# --- 경로 설정 및 하이퍼파라미터 ---
BASE_DIR     = f"./data/covid19-xray-severity-scoring/"
CSV_PATH     = str(Path(BASE_DIR) / "Brixia.csv")
IMAGE_DIR    = str(Path(BASE_DIR) / "segmented_png")

OUT_DIR      = "./runs_severity_classification"
BEST_PATH    = str(Path(OUT_DIR) / "best_efficientnet_b0_classification.pth")
os.makedirs(OUT_DIR, exist_ok=True)

DEVICE       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED         = 42
IMG_SIZE     = 224
BATCH_SIZE   = 32
NUM_CLASSES  = 4   # 0, 1, 2, 3
EPOCHS       = 100  # Single phase training
LR           = 1e-4
WEIGHT_DECAY = 5e-4
AMP          = True
EARLY_STOP_ACC = 0.75  # 🔄 MAE → Accuracy
DROP_RATIO   = 0.3
AUG_RATIO    = 0.5
MIXUP_ALPHA  = 0.2
LABEL_SMOOTHING = 0.1  # ✅ NEW: Label smoothing

# --- 시드 고정 ---
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
set_seed(SEED)

def make_transform_with_label(train: bool, img_size: int = IMG_SIZE, aug_ratio=AUG_RATIO):
    """Brixia Score의 좌우 구조를 고려한 transform"""
    def _tfm(img: Image.Image, label: torch.Tensor = None):
        img = img.convert('RGB')
        img = TF.resize(
            img, 
            [img_size, img_size], 
            interpolation=TF.InterpolationMode.BILINEAR,
            antialias=True
        )
        
        if train:
            # 1. Horizontal Flip (좌우 반전: ABC ↔ DEF)
            if random.random() < aug_ratio:
                img = TF.hflip(img)
                if label is not None:
                    # [A, B, C, D, E, F] → [D, E, F, A, B, C]
                    label = label[[3, 4, 5, 0, 1, 2]]
            
            # 2. 약한 회전 (±5도)
            if random.random() < aug_ratio:
                angle = float(torch.empty(1).uniform_(-5, 5))
                img = TF.rotate(
                    img, 
                    angle, 
                    interpolation=TF.InterpolationMode.BILINEAR,
                    fill=0
                )
            
            # 3. 약한 Translation
            if random.random() < aug_ratio:
                max_dx = 0.05 * img_size
                max_dy = 0.05 * img_size
                translations = (
                    float(torch.empty(1).uniform_(-max_dx, max_dx)),
                    float(torch.empty(1).uniform_(-max_dy, max_dy))
                )
                img = TF.affine(
                    img,
                    angle=0,
                    translate=translations,
                    scale=1.0,
                    shear=0,
                    interpolation=TF.InterpolationMode.BILINEAR,
                    fill=0
                )
            
            # 4. Brightness & Contrast
            if random.random() < aug_ratio:
                brightness_factor = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_brightness(img, brightness_factor)
            
            if random.random() < aug_ratio:
                contrast_factor = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_contrast(img, contrast_factor)
            
            # 5. Gamma Correction
            if random.random() < 0.3:
                gamma = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_gamma(img, gamma)
        
        # Tensor 변환
        img = TF.to_tensor(img)
        
        # Gaussian Noise (train only)
        if train and random.random() < 0.2:
            noise = torch.randn_like(img) * 0.01
            img = img + noise
            img = torch.clamp(img, 0, 1)
        
        # 정규화
        img = TF.normalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        if label is not None:
            return img, label
        return img
    
    return _tfm

def load_and_split_brixia(csv_path, val_ratio=0.2, seed=SEED):
    df = pd.read_csv(csv_path, dtype={'BrixiaScore': str})
    df = df.dropna(subset=['BrixiaScore'])
    df = df[df['BrixiaScore'] != 'nan']
    df = df[df['BrixiaScore'].str.len() == 6].copy()
    
    print(f"유효한 데이터: {len(df)}개")
    
    if 'ConsensusTestset' in df.columns:
        test_df = df[df['ConsensusTestset'] == 1].copy()
        train_val_df = df[df['ConsensusTestset'] == 0].copy()
    else:
        test_df = pd.DataFrame()
        train_val_df = df.copy()
    
    gss = GroupShuffleSplit(n_splits=1, test_size=val_ratio, random_state=seed)
    train_idx, val_idx = next(gss.split(
        train_val_df, 
        groups=train_val_df['StudyId']
    ))
    
    tr_df = train_val_df.iloc[train_idx].copy()
    val_df = train_val_df.iloc[val_idx].copy()
    
    print(f"Train: {len(tr_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
    validate_split(tr_df, val_df, test_df)
    
    return tr_df, val_df, test_df

def validate_split(tr_df, val_df, tt_df):
    train_studies = set(tr_df['StudyId'])
    val_studies = set(val_df['StudyId'])
    test_studies = set(tt_df['StudyId']) if len(tt_df) > 0 else set()
    
    assert len(train_studies & val_studies) == 0, "Train-Val 간 StudyId 중복!"
    assert len(train_studies & test_studies) == 0, "Train-Test 간 StudyId 중복!"
    assert len(val_studies & test_studies) == 0, "Val-Test 간 StudyId 중복!"
    
    for name, data in [('Train', tr_df), ('Val', val_df), ('Test', tt_df)]:
        if len(data) > 0:
            scores = data['BrixiaScore'].apply(lambda x: sum(int(c) for c in x))
            print(f"{name} - Mean: {scores.mean():.2f}, Std: {scores.std():.2f}")
    
    return True

# ============================================================
# Dataset
# ============================================================
class BrixiaDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.img_col = "Filename"
        self.label_col = "BrixiaScore"
        self._validate_data()
    
    def _validate_data(self):
        assert self.img_col in self.df.columns
        assert self.label_col in self.df.columns
        
        invalid_scores = self.df[self.df[self.label_col].str.len() != 6]
        if len(invalid_scores) > 0:
            print(f"⚠️ 경고: {len(invalid_scores)}개의 잘못된 BrixiaScore 발견")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        img_name_from_csv = row[self.img_col]
        img_name = img_name_from_csv.replace('.dcm', '.png')
        img_path = os.path.join(self.img_dir, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"❌ 이미지 로드 오류: {img_path}")
            raise
        
        scores_str = row[self.label_col]
        scores_list = [int(c) for c in scores_str]
        labels = torch.tensor(scores_list, dtype=torch.long)  # 🔄 long으로 변경
        
        if self.transform:
            image, labels = self.transform(image, labels)
        else:
            image = TF.to_tensor(image)
            image = TF.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, labels

def create_dataloaders(tr_df, val_df, tt_df, img_dir, 
                       batch_size=32, img_size=224, num_workers=4):
    train_transform = make_transform_with_label(train=True, img_size=img_size)
    val_transform = make_transform_with_label(train=False, img_size=img_size)
    
    tr_ds = BrixiaDataset(tr_df, img_dir, transform=train_transform)
    val_ds = BrixiaDataset(val_df, img_dir, transform=val_transform)
    tt_ds = BrixiaDataset(tt_df, img_dir, transform=val_transform)
    
    tr_loader = DataLoader(
        tr_ds, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available(),
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_ds, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )
    
    tt_loader = DataLoader(
        tt_ds, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )
    
    print("✅ DataLoader 준비 완료")
    print(f"   Train: {len(tr_ds)} samples, {len(tr_loader)} batches")
    print(f"   Val:   {len(val_ds)} samples, {len(val_loader)} batches")
    print(f"   Test:  {len(tt_ds)} samples, {len(tt_loader)} batches")
    
    return tr_loader, val_loader, tt_loader

# ============================================================
# Loss Function (분류용으로 변경)
# ============================================================
def calculate_class_weights(labels, num_classes=4, method='sqrt_inverse'):
    """클래스 불균형 해결을 위한 가중치 계산"""
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    labels_flat = labels.flatten()
    counts = np.bincount(labels_flat.astype(int), minlength=num_classes)
    
    if method == 'sqrt_inverse':
        weights = 1.0 / (np.sqrt(counts) + 1e-6)
    elif method == 'inverse':
        weights = 1.0 / (counts + 1e-6)
    else:
        total = len(labels_flat)
        weights = total / (num_classes * (counts + 1e-6))
    
    weights = weights / weights.mean()
    return torch.FloatTensor(weights)

def print_class_distribution(labels):
    """클래스 분포 시각화"""
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    print("=" * 60)
    print("Class Distribution Analysis")
    print("=" * 60)
    
    region_names = ['A', 'B', 'C', 'D', 'E', 'F']
    for idx, name in enumerate(region_names):
        region_labels = labels[:, idx]
        counts = np.bincount(region_labels.astype(int), minlength=4)
        total = counts.sum()
        
        print(f"\n{name}:")
        for cls in range(4):
            pct = 100 * counts[cls] / total if total > 0 else 0
            bar = '█' * int(pct / 2)
            print(f"  Class {cls}: {counts[cls]:4d} ({pct:5.1f}%) {bar}")
    
    print("=" * 60)

class AdaptiveClassificationLoss(nn.Module):
    """
    ✅ NEW: 다중 클래스 분류를 위한 손실함수
    - CrossEntropyLoss 기반
    - 클래스 가중치 적용
    - 부위별 가중치 적용
    - Label Smoothing 지원
    """
    
    def __init__(self, train_labels, num_classes=4, use_class_weights=True, 
                 part_weights=None, label_smoothing=0.1):
        super().__init__()
        self.num_classes = num_classes
        self.label_smoothing = label_smoothing
        
        if isinstance(train_labels, torch.Tensor):
            train_labels_np = train_labels.cpu().numpy()
        else:
            train_labels_np = train_labels
        
        print_class_distribution(train_labels_np)
        
        # 클래스 가중치
        if use_class_weights:
            class_weights = calculate_class_weights(train_labels_np, num_classes=num_classes)
            self.register_buffer('class_weights', class_weights)
            print(f"✅ Class weights: {class_weights.numpy()}")
        else:
            self.class_weights = None
        
        # 부위별 가중치
        if part_weights is None:
            self.part_weights = torch.ones(6)
        else:
            self.part_weights = torch.tensor(part_weights, dtype=torch.float32)
        self.register_buffer('part_weights_buf', self.part_weights)
        print(f"✅ Part weights: {self.part_weights.numpy()}")

    def forward(self, pred_logits, target, use_mixup=False):
        """
        pred_logits: [B, 6, 4] - 각 부위별 4개 클래스에 대한 logits
        target: [B, 6] - 각 부위별 클래스 레이블 (0~3)
        """
        B, num_regions, num_classes = pred_logits.shape
        
        # Reshape for loss computation
        pred_logits_flat = pred_logits.view(B * num_regions, num_classes)  # [B*6, 4]
        target_flat = target.view(B * num_regions)  # [B*6]
        
        # CrossEntropyLoss with label smoothing
        if self.class_weights is not None and not use_mixup:
            criterion = nn.CrossEntropyLoss(
                weight=self.class_weights.to(pred_logits.device),
                label_smoothing=self.label_smoothing,
                reduction='none'
            )
        else:
            criterion = nn.CrossEntropyLoss(
                label_smoothing=self.label_smoothing,
                reduction='none'
            )
        
        loss = criterion(pred_logits_flat, target_flat)  # [B*6]
        loss = loss.view(B, num_regions)  # [B, 6]
        
        # 부위별 가중치 적용
        part_weights = self.part_weights_buf.to(pred_logits.device)
        loss = loss * part_weights.unsqueeze(0)  # [B, 6]
        
        # 부위별 평균 손실 (모니터링용)
        part_losses = loss.mean(dim=0)  # [6]
        
        return loss.mean(), part_losses

# ============================================================
# Mixup (분류용)
# ============================================================
def mixup_data_classification(x, y, alpha=MIXUP_ALPHA):
    """
    ✅ NEW: 분류용 Mixup
    y는 one-hot으로 변환 후 mixup
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    
    # One-hot encoding
    y_onehot = F.one_hot(y, num_classes=NUM_CLASSES).float()  # [B, 6, 4]
    y_onehot_shuffled = y_onehot[index]
    
    mixed_y = lam * y_onehot + (1 - lam) * y_onehot_shuffled  # [B, 6, 4]
    
    return mixed_x, mixed_y, lam

def mixup_criterion(criterion, pred_logits, y_mixed, lam):
    """Mixup을 위한 손실 계산"""
    # y_mixed는 [B, 6, 4] soft labels
    # pred_logits는 [B, 6, 4]
    
    B, num_regions, num_classes = pred_logits.shape
    
    # Reshape
    pred_flat = pred_logits.view(B * num_regions, num_classes)  # [B*6, 4]
    target_flat = y_mixed.view(B * num_regions, num_classes)  # [B*6, 4]
    
    # Soft target loss
    log_probs = F.log_softmax(pred_flat, dim=1)
    loss = -(target_flat * log_probs).sum(dim=1)  # [B*6]
    loss = loss.view(B, num_regions)  # [B, 6]
    
    # 부위별 가중치 적용 (criterion에서 가져옴)
    if hasattr(criterion, 'part_weights_buf'):
        part_weights = criterion.part_weights_buf.to(pred_logits.device)
        loss = loss * part_weights.unsqueeze(0)
    
    part_losses = loss.mean(dim=0)
    return loss.mean(), part_losses

# ============================================================
# Metrics (분류용)
# ============================================================
@torch.no_grad()
def calculate_classification_metrics(pred_logits, labels):
    """
    ✅ NEW: 분류 지표 계산
    pred_logits: [B, 6, 4]
    labels: [B, 6]
    """
    # 예측 클래스
    preds = pred_logits.argmax(dim=-1)  # [B, 6]
    
    # Exact match accuracy
    exact_acc = (preds == labels).float().mean().item()
    
    # Off-by-1 accuracy (인접 클래스 허용)
    off_by_1 = (torch.abs(preds - labels) <= 1).float().mean().item()
    
    # Per-region accuracy
    region_acc = (preds == labels).float().mean(dim=0)  # [6]
    
    # MAE (참고용)
    mae = torch.abs(preds.float() - labels.float()).mean().item()
    
    return exact_acc, off_by_1, mae, region_acc

# ============================================================
# Model (분류용으로 변경)
# ============================================================
class EfficientNetB0Classification(nn.Module):
    """
    ✅ 분류를 위한 모델 (부위 간 상관관계 고려)
    각 부위(A~F)마다 4개 클래스(0~3) 예측
    + Self-Attention으로 부위 간 관계 학습
    """
    def __init__(self, pretrained=True, drop=0.3, num_regions=6, num_classes=4):
        super().__init__()
        
        weights = EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None
        backbone = efficientnet_b0(weights=weights)
        
        self.features = backbone.features
        in_feat = 1280
        
        # Positional embedding
        self.pos_embed = nn.Parameter(torch.randn(1, 49, in_feat))
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        
        # Region queries
        self.region_queries = nn.Parameter(torch.randn(num_regions, in_feat))
        nn.init.xavier_uniform_(self.region_queries)
        
        # Cross attention (이미지 특징 → 부위별 특징)
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=in_feat,
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )
        
        self.norm1 = nn.LayerNorm(in_feat)
        
        # ✅ NEW: Self-Attention (부위 간 상관관계 학습)
        self.self_attention = nn.MultiheadAttention(
            embed_dim=in_feat,
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )
        
        self.norm2 = nn.LayerNorm(in_feat)
        self.norm3 = nn.LayerNorm(in_feat)
        
        # FFN
        self.ffn = nn.Sequential(
            nn.Linear(in_feat, in_feat * 2),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(in_feat * 2, in_feat),
            nn.Dropout(drop)
        )
        
        self.norm4 = nn.LayerNorm(in_feat)
        
        # Classification heads: 각 부위마다 4개 클래스 예측
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(in_feat, 256),
                nn.LayerNorm(256),
                nn.GELU(),
                nn.Dropout(drop),
                nn.Linear(256, 128),
                nn.LayerNorm(128),
                nn.GELU(),
                nn.Dropout(drop),
                nn.Linear(128, num_classes)
            ) for _ in range(num_regions)
        ])
    
    def forward(self, x):
        B = x.size(0)
        
        # Feature extraction
        feat = self.features(x)  # [B, 1280, 7, 7]
        feat = feat.flatten(2).transpose(1, 2)  # [B, 49, 1280]
        feat = feat + self.pos_embed
        
        # Region queries
        queries = self.region_queries.unsqueeze(0).expand(B, -1, -1)  # [B, 6, 1280]
        
        # Cross attention (이미지 → 부위)
        attn_out, _ = self.cross_attention(
            query=queries,
            key=feat,
            value=feat
        )
        attn_out = self.norm1(attn_out + queries)  # [B, 6, 1280]
        
        # ✅ Self-Attention (부위 간 상관관계)
        # A, B, C가 서로 영향을 주고, D, E, F도 서로 영향
        self_attn_out, attn_weights = self.self_attention(
            query=attn_out,
            key=attn_out,
            value=attn_out
        )
        attn_out = self.norm2(attn_out + self_attn_out)  # [B, 6, 1280]
        
        # FFN
        ffn_out = self.ffn(attn_out)
        attn_out = self.norm3(attn_out + ffn_out)  # [B, 6, 1280]
        
        # 각 부위별 classification head 통과
        outputs = []
        for i in range(len(self.heads)):
            region_feat = attn_out[:, i, :]  # [B, 1280]
            logits = self.heads[i](region_feat)  # [B, 4]
            outputs.append(logits)
        
        out = torch.stack(outputs, dim=1)  # [B, 6, 4]
        return out, attn_weights  # ✅ attention weights 반환 (Grad-CAM용)

# ============================================================
# Training Functions (분류용 수정)
# ============================================================
def train_epoch(model, tr_loader, criterion, optimizer, scaler, device, 
                amp=True, use_mixup=True):
    model.train()
    run_loss = run_acc = run_off1 = run_mae = n = 0
    part_losses_sum = torch.zeros(6)
    
    pbar = tqdm(tr_loader, desc="Train", leave=False)
    
    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)  # [B, 6]
        
        # Mixup 적용
        is_mixup = use_mixup and (random.random() < 0.5)
        
        optimizer.zero_grad(set_to_none=True)
        
        with autocast(enabled=amp):
            if is_mixup:
                imgs_mixed, labels_mixed, lam = mixup_data_classification(imgs, labels)
                pred_logits, _ = model(imgs_mixed)  # attention weights 무시
                loss, part_losses = mixup_criterion(criterion, pred_logits, labels_mixed, lam)
            else:
                pred_logits, _ = model(imgs)  # attention weights 무시
                loss, part_losses = criterion(pred_logits, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Metrics (원본 labels로 계산)
        exact_acc, off_by_1, mae, _ = calculate_classification_metrics(pred_logits.detach(), labels)
        
        bs = imgs.size(0)
        run_loss += loss.item() * bs
        run_acc += exact_acc * bs
        run_off1 += off_by_1 * bs
        run_mae += mae * bs
        part_losses_sum += part_losses.cpu() * bs
        n += bs
        
        pbar.set_postfix(
            loss=f"{run_loss/n:.4f}",
            acc=f"{run_acc/n:.4f}",
            mae=f"{run_mae/n:.4f}"
        )
    
    part_losses_avg = part_losses_sum / n
    
    return run_loss/n, run_acc/n, run_off1/n, run_mae/n, part_losses_avg

@torch.no_grad()
def evaluate(model, val_loader, criterion, device, split='val'):
    model.eval()
    run_loss = run_acc = run_off1 = run_mae = n = 0
    part_losses_sum = torch.zeros(6)
    region_acc_sum = torch.zeros(6)
    
    pbar = tqdm(val_loader, desc=f"{split.capitalize()}", leave=False)
    
    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        pred_logits, _ = model(imgs)  # [B, 6, 4], attention weights 무시
        loss, part_losses = criterion(pred_logits, labels)
        
        exact_acc, off_by_1, mae, region_acc = calculate_classification_metrics(pred_logits, labels)
        
        bs = imgs.size(0)
        run_loss += loss.item() * bs
        run_acc += exact_acc * bs
        run_off1 += off_by_1 * bs
        run_mae += mae * bs
        part_losses_sum += part_losses.cpu() * bs
        region_acc_sum += region_acc.cpu() * bs
        n += bs
        
        pbar.set_postfix(
            loss=f"{run_loss/n:.4f}",
            acc=f"{run_acc/n:.4f}"
        )
    
    avg_loss = run_loss/n
    avg_acc = run_acc/n
    avg_off1 = run_off1/n
    avg_mae = run_mae/n
    part_losses_avg = part_losses_sum / n
    region_acc_avg = region_acc_sum / n
    
    print(f"[{split}] loss:{avg_loss:.4f} acc:{avg_acc:.4f} "
          f"off1:{avg_off1:.4f} mae:{avg_mae:.4f}")
    print(f"  Region Acc: {region_acc_avg.numpy().round(3)}")
    
    return avg_loss, avg_acc, avg_off1, avg_mae, part_losses_avg, region_acc_avg

def get_lrs(optimizer):
    return [pg['lr'] for pg in optimizer.param_groups]

# ============================================================
# Main Function
# ============================================================
def main():
    print("\n" + "="*70)
    print("🚀 Brixia COVID-19 Classification Training")
    print("   ✅ 클래스 불균형 처리 (자동 가중치)")
    print("   ✅ 부위 간 상관관계 학습 (Self-Attention)")
    print("="*70)
    
    # 데이터 준비
    print("\n📂 Loading data...")
    tr_df, val_df, tt_df = load_and_split_brixia(CSV_PATH)
    
    print("\n📦 Creating DataLoaders...")
    tr_loader, val_loader, tt_loader = create_dataloaders(
        tr_df, val_df, tt_df, img_dir=IMAGE_DIR, 
        batch_size=BATCH_SIZE, img_size=IMG_SIZE, num_workers=4
    )
    
    # Train labels 추출
    train_labels = torch.cat([labels for _, labels in tr_loader], dim=0)
    
    # ========================================
    # 학습 준비
    # ========================================
    print("\n" + "="*70)
    print("📍 Training Setup")
    print("="*70)
    
    # ✅ 클래스 불균형 자동 처리
    criterion = AdaptiveClassificationLoss(
        train_labels, 
        num_classes=NUM_CLASSES,
        use_class_weights=True,  # 클래스 가중치 자동 계산
        part_weights=None,  # 부위별 균등 가중치
        label_smoothing=LABEL_SMOOTHING
    )
    
    # ✅ 부위 간 상관관계를 고려하는 모델
    model = EfficientNetB0Classification(
        pretrained=True, 
        drop=DROP_RATIO,
        num_regions=6,
        num_classes=NUM_CLASSES
    ).to(DEVICE)
    print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, min_lr=1e-6
    )
    scaler = GradScaler(enabled=AMP)
    
    best_acc = 0.0
    best_mae = float('inf')
    patience_counter = 0
    max_patience = 15
    
    # ========================================
    # 학습 루프
    # ========================================
    print("\n" + "="*70)
    print("🏋️ Training Start")
    print("="*70)
    
    for ep in range(1, EPOCHS + 1):
        t0 = time.time()
        
        tr_loss, tr_acc, tr_off1, tr_mae, tr_part_losses = train_epoch(
            model, tr_loader, criterion, optimizer, scaler, DEVICE, AMP
        )
        
        val_loss, val_acc, val_off1, val_mae, val_part_losses, val_region_acc = evaluate(
            model, val_loader, criterion, DEVICE, split='val'
        )
        
        scheduler.step(val_acc)
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            best_mae = val_mae
            patience_counter = 0
            torch.save({
                'epoch': ep,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': best_acc,
                'val_mae': best_mae,
                'val_off1': val_off1,
                'region_acc': val_region_acc,
            }, BEST_PATH)
            print(f"✅ New Best! (Acc={best_acc:.4f}, MAE={best_mae:.4f})")
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= max_patience:
            print(f"\n⏹️ Early stopping at epoch {ep}")
            break
        
        elapsed = time.time() - t0
        print(f"\n[Epoch {ep:03d}/{EPOCHS}]")
        print(f"  Train - loss:{tr_loss:.4f} acc:{tr_acc:.4f} off1:{tr_off1:.4f} mae:{tr_mae:.4f}")
        print(f"  Val   - loss:{val_loss:.4f} acc:{val_acc:.4f} off1:{val_off1:.4f} mae:{val_mae:.4f}")
        print(f"  Part losses (Val): {val_part_losses.numpy().round(3)}")
        print(f"  Region Acc (Val): {val_region_acc.numpy().round(3)}")
        print(f"  LR:{get_lrs(optimizer)[0]:.2e} | {elapsed:.1f}s | Patience:{patience_counter}/{max_patience}")
        print("-" * 70)
    
    # ========================================
    # Test Evaluation
    # ========================================
    print("\n" + "="*70)
    print("🎉 Training Finished!")
    print("="*70)
    
    if len(tt_loader) > 0:
        print("\n📊 Test evaluation with best model...")
        checkpoint = torch.load(BEST_PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        
        tt_loss, tt_acc, tt_off1, tt_mae, tt_part_losses, tt_region_acc = evaluate(
            model, tt_loader, criterion, DEVICE, split='test'
        )
        
        print(f"\n🏆 Test Results:")
        print(f"   Accuracy: {tt_acc:.4f}")
        print(f"   Off-by-1: {tt_off1:.4f}")
        print(f"   MAE: {tt_mae:.4f}")
        print(f"   Region Acc: {tt_region_acc.numpy().round(3)}")
        print(f"   Part losses: {tt_part_losses.numpy().round(3)}")
    
    print(f"\n💾 Best model saved: {BEST_PATH}")
    print(f"📈 Best Validation Accuracy: {best_acc:.4f}")
    print(f"📉 Best Validation MAE: {best_mae:.4f}")
    print("\n✅ 이제 gradcam_inference.py를 실행하여 결과를 시각화하세요!")
    print("="*70)

if __name__ == "__main__":
    main()


🚀 Brixia COVID-19 Classification Training
   ✅ 클래스 불균형 처리 (자동 가중치)
   ✅ 부위 간 상관관계 학습 (Self-Attention)

📂 Loading data...
유효한 데이터: 4695개
Train: 3637, Val: 912, Test: 146
Train - Mean: 8.31, Std: 4.26
Val - Mean: 8.35, Std: 4.15
Test - Mean: 7.78, Std: 4.20

📦 Creating DataLoaders...
✅ DataLoader 준비 완료
   Train: 3637 samples, 113 batches
   Val:   912 samples, 29 batches
   Test:  146 samples, 5 batches

📍 Training Setup
Class Distribution Analysis

A:
  Class 0: 1810 ( 50.1%) █████████████████████████
  Class 1: 1126 ( 31.1%) ███████████████
  Class 2:  446 ( 12.3%) ██████
  Class 3:  234 (  6.5%) ███

B:
  Class 0:  721 ( 19.9%) █████████
  Class 1:  949 ( 26.2%) █████████████
  Class 2: 1171 ( 32.4%) ████████████████
  Class 3:  775 ( 21.4%) ██████████

C:
  Class 0:  406 ( 11.2%) █████
  Class 1:  783 ( 21.7%) ██████████
  Class 2: 1300 ( 36.0%) █████████████████
  Class 3: 1127 ( 31.2%) ███████████████

D:
  Class 0: 1804 ( 49.9%) ████████████████████████
  Class 1: 1126 ( 31.1%) █

[val] loss:1.1561 acc:0.4856 off1:0.8688 mae:0.6603
  Region Acc: [0.542 0.44  0.405 0.617 0.467 0.443]
✅ New Best! (Acc=0.4856, MAE=0.6603)

[Epoch 001/100]
  Train - loss:1.2178 acc:0.4151 off1:0.8294 mae:0.7886
  Val   - loss:1.1561 acc:0.4856 off1:0.8688 mae:0.6603
  Part losses (Val): [1.141 1.203 1.239 0.984 1.166 1.203]
  Region Acc (Val): [0.542 0.44  0.405 0.617 0.467 0.443]
  LR:1.00e-04 | 15.4s | Patience:0/15
----------------------------------------------------------------------


[val] loss:1.1078 acc:0.5143 off1:0.9123 mae:0.5780
  Region Acc: [0.546 0.489 0.463 0.612 0.505 0.47 ]
✅ New Best! (Acc=0.5143, MAE=0.5780)

[Epoch 002/100]
  Train - loss:1.1414 acc:0.4534 off1:0.8569 mae:0.7148
  Val   - loss:1.1078 acc:0.5143 off1:0.9123 mae:0.5780
  Part losses (Val): [1.073 1.14  1.175 0.969 1.119 1.171]
  Region Acc (Val): [0.546 0.489 0.463 0.612 0.505 0.47 ]
  LR:1.00e-04 | 14.9s | Patience:0/15
----------------------------------------------------------------------


[val] loss:1.0821 acc:0.5521 off1:0.9240 mae:0.5307
  Region Acc: [0.582 0.518 0.514 0.654 0.534 0.511]
✅ New Best! (Acc=0.5521, MAE=0.5307)

[Epoch 003/100]
  Train - loss:1.1075 acc:0.4791 off1:0.8797 mae:0.6592
  Val   - loss:1.0821 acc:0.5521 off1:0.9240 mae:0.5307
  Part losses (Val): [1.057 1.125 1.132 0.941 1.096 1.141]
  Region Acc (Val): [0.582 0.518 0.514 0.654 0.534 0.511]
  LR:1.00e-04 | 15.0s | Patience:0/15
----------------------------------------------------------------------


[val] loss:1.0551 acc:0.5523 off1:0.9390 mae:0.5143
  Region Acc: [0.557 0.541 0.534 0.621 0.545 0.516]
✅ New Best! (Acc=0.5523, MAE=0.5143)

[Epoch 004/100]
  Train - loss:1.0675 acc:0.4775 off1:0.8786 mae:0.6667
  Val   - loss:1.0551 acc:0.5523 off1:0.9390 mae:0.5143
  Part losses (Val): [1.04  1.072 1.093 0.944 1.059 1.122]
  Region Acc (Val): [0.557 0.541 0.534 0.621 0.545 0.516]
  LR:1.00e-04 | 15.4s | Patience:0/15
----------------------------------------------------------------------


...

[val] loss:1.0966 acc:0.5830 off1:0.9545 mae:0.4660
  Region Acc: [0.615 0.56  0.559 0.679 0.573 0.511]

[Epoch 023/100]
  Train - loss:0.8471 acc:0.5946 off1:0.9120 mae:0.5131
  Val   - loss:1.0966 acc:0.5830 off1:0.9545 mae:0.4660
  Part losses (Val): [1.094 1.089 1.113 0.989 1.104 1.191]
  Region Acc (Val): [0.615 0.56  0.559 0.679 0.573 0.511]
  LR:2.50e-05 | 13.9s | Patience:12/15
----------------------------------------------------------------------


[val] loss:1.0951 acc:0.5808 off1:0.9547 mae:0.4678
  Region Acc: [0.616 0.566 0.565 0.664 0.565 0.509]

[Epoch 024/100]
  Train - loss:0.8297 acc:0.6269 off1:0.9249 mae:0.4644
  Val   - loss:1.0951 acc:0.5808 off1:0.9547 mae:0.4678
  Part losses (Val): [1.074 1.094 1.131 0.968 1.106 1.198]
  Region Acc (Val): [0.616 0.566 0.565 0.664 0.565 0.509]
  LR:2.50e-05 | 15.0s | Patience:13/15
----------------------------------------------------------------------


[val] loss:1.0956 acc:0.5810 off1:0.9576 mae:0.4653
  Region Acc: [0.606 0.562 0.557 0.673 0.575 0.512]

[Epoch 025/100]
  Train - loss:0.7912 acc:0.6228 off1:0.9186 mae:0.4758
  Val   - loss:1.0956 acc:0.5810 off1:0.9576 mae:0.4653
  Part losses (Val): [1.064 1.108 1.137 0.971 1.089 1.205]
  Region Acc (Val): [0.606 0.562 0.557 0.673 0.575 0.512]
  LR:2.50e-05 | 14.4s | Patience:14/15
----------------------------------------------------------------------


[val] loss:1.0993 acc:0.5833 off1:0.9543 mae:0.4656
  Region Acc: [0.621 0.562 0.557 0.673 0.562 0.524]

⏹️ Early stopping at epoch 26

🎉 Training Finished!

📊 Test evaluation with best model...


[test] loss:1.0082 acc:0.5868 off1:0.9646 mae:0.4521
  Region Acc: [0.616 0.521 0.596 0.671 0.555 0.562]

🏆 Test Results:
   Accuracy: 0.5868
   Off-by-1: 0.9646
   MAE: 0.4521
   Region Acc: [0.616 0.521 0.596 0.671 0.555 0.562]
   Part losses: [0.937 1.083 1.057 0.859 1.013 1.1  ]

💾 Best model saved: runs_severity_classification/best_efficientnet_b0_classification.pth
📈 Best Validation Accuracy: 0.5872
📉 Best Validation MAE: 0.4629

✅ 이제 gradcam_inference.py를 실행하여 결과를 시각화하세요!


In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import functional as TF
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# ============================================================
# 설정
# ============================================================
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_SIZE = 224
NUM_CLASSES = 4
NUM_REGIONS = 6

MODEL_PATH = "./runs_severity_classification/best_efficientnet_b0_classification.pth"
OUTPUT_DIR = "./gradcam_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

REGION_NAMES = ['A (좌상)', 'B (좌중)', 'C (좌하)', 'D (우상)', 'E (우중)', 'F (우하)']
CLASS_NAMES = ['정상 (0)', '경증 (1)', '중등도 (2)', '중증 (3)']

# 부위별 색상 (시각화용)
REGION_COLORS = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#F7DC6F']

# ============================================================
# 모델 정의 (학습 코드와 동일)
# ============================================================
class EfficientNetB0Classification(nn.Module):
    def __init__(self, pretrained=False, drop=0.3, num_regions=6, num_classes=4):
        super().__init__()
        
        weights = EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None
        backbone = efficientnet_b0(weights=weights)
        
        self.features = backbone.features
        in_feat = 1280
        
        self.pos_embed = nn.Parameter(torch.randn(1, 49, in_feat))
        self.region_queries = nn.Parameter(torch.randn(num_regions, in_feat))
        
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=in_feat, num_heads=8, dropout=0.1, batch_first=True
        )
        self.norm1 = nn.LayerNorm(in_feat)
        
        self.self_attention = nn.MultiheadAttention(
            embed_dim=in_feat, num_heads=8, dropout=0.1, batch_first=True
        )
        self.norm2 = nn.LayerNorm(in_feat)
        self.norm3 = nn.LayerNorm(in_feat)
        
        self.ffn = nn.Sequential(
            nn.Linear(in_feat, in_feat * 2),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(in_feat * 2, in_feat),
            nn.Dropout(drop)
        )
        self.norm4 = nn.LayerNorm(in_feat)
        
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(in_feat, 256),
                nn.LayerNorm(256),
                nn.GELU(),
                nn.Dropout(drop),
                nn.Linear(256, 128),
                nn.LayerNorm(128),
                nn.GELU(),
                nn.Dropout(drop),
                nn.Linear(128, num_classes)
            ) for _ in range(num_regions)
        ])
    
    def forward(self, x):
        B = x.size(0)
        feat = self.features(x)
        feat = feat.flatten(2).transpose(1, 2)
        feat = feat + self.pos_embed
        
        queries = self.region_queries.unsqueeze(0).expand(B, -1, -1)
        attn_out, _ = self.cross_attention(query=queries, key=feat, value=feat)
        attn_out = self.norm1(attn_out + queries)
        
        self_attn_out, attn_weights = self.self_attention(
            query=attn_out, key=attn_out, value=attn_out
        )
        attn_out = self.norm2(attn_out + self_attn_out)
        ffn_out = self.ffn(attn_out)
        attn_out = self.norm3(attn_out + ffn_out)
        
        outputs = []
        for i in range(len(self.heads)):
            region_feat = attn_out[:, i, :]
            logits = self.heads[i](region_feat)
            outputs.append(logits)
        
        out = torch.stack(outputs, dim=1)
        return out, attn_weights

# ============================================================
# Grad-CAM 구현
# ============================================================
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Hook 등록
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_full_backward_hook(self.save_gradient)
    
    def save_activation(self, module, input, output):
        self.activations = output.detach()
    
    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
    
    def generate_cam(self, input_image, region_idx, class_idx=None):
        """
        특정 부위(region_idx)와 클래스(class_idx)에 대한 CAM 생성
        """
        self.model.eval()
        
        # Forward pass
        output, _ = self.model(input_image)  # [1, 6, 4]
        
        # 특정 부위의 예측
        region_output = output[0, region_idx, :]  # [4]
        
        # class_idx가 없으면 예측된 클래스 사용
        if class_idx is None:
            class_idx = region_output.argmax().item()
        
        # Backward pass
        self.model.zero_grad()
        target = region_output[class_idx]
        target.backward(retain_graph=True)
        
        # Grad-CAM 계산
        gradients = self.gradients[0]  # [C, H, W]
        activations = self.activations[0]  # [C, H, W]
        
        # Global Average Pooling on gradients
        weights = gradients.mean(dim=(1, 2), keepdim=True)  # [C, 1, 1]
        
        # Weighted combination
        cam = (weights * activations).sum(dim=0)  # [H, W]
        cam = F.relu(cam)
        
        # Normalize
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)
        
        return cam.cpu().numpy(), class_idx

# ============================================================
# 시각화 함수
# ============================================================
def preprocess_image(image_path, img_size=IMG_SIZE):
    """이미지 전처리"""
    img = Image.open(image_path).convert('RGB')
    img_resized = img.resize((img_size, img_size), Image.BILINEAR)
    
    # Tensor 변환
    img_tensor = TF.to_tensor(img_resized)
    img_tensor = TF.normalize(img_tensor, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    
    return img_tensor.unsqueeze(0), img_resized

def visualize_gradcam(original_img, cam, region_name, pred_class, confidence, 
                      save_path=None, alpha=0.4):
    """Grad-CAM 히트맵 시각화"""
    # CAM을 원본 이미지 크기로 리사이즈
    cam_resized = np.array(Image.fromarray(cam).resize(
        original_img.size, Image.BILINEAR
    ))
    
    # 히트맵 생성
    plt.figure(figsize=(10, 8))
    
    # 원본 이미지
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    plt.title(f'{region_name}\n예측: {CLASS_NAMES[pred_class]} ({confidence:.1f}%)', 
              fontsize=12, fontweight='bold')
    plt.axis('off')
    
    # Grad-CAM 오버레이
    plt.subplot(1, 2, 2)
    plt.imshow(original_img)
    plt.imshow(cam_resized, cmap='jet', alpha=alpha)
    plt.title(f'Grad-CAM Heatmap', fontsize=12, fontweight='bold')
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✅ 저장: {save_path}")
    
    plt.close()

def visualize_all_regions(original_img, model, img_tensor, predictions, 
                          confidences, save_path=None):
    """6개 부위 모두 시각화 (2x3 grid)"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    # Grad-CAM 생성
    target_layer = model.features[-1]  # EfficientNet 마지막 Conv layer
    grad_cam = GradCAM(model, target_layer)
    
    for idx in range(6):
        cam, pred_class = grad_cam.generate_cam(img_tensor, region_idx=idx)
        
        # CAM 리사이즈
        cam_resized = np.array(Image.fromarray(cam).resize(
            original_img.size, Image.BILINEAR
        ))
        
        # 시각화
        axes[idx].imshow(original_img)
        im = axes[idx].imshow(cam_resized, cmap='jet', alpha=0.4)
        
        axes[idx].set_title(
            f'{REGION_NAMES[idx]}\n{CLASS_NAMES[pred_class]} ({confidences[idx]:.1f}%)',
            fontsize=12, fontweight='bold', color=REGION_COLORS[idx]
        )
        axes[idx].axis('off')
        
        # Colorbar
        cbar = plt.colorbar(im, ax=axes[idx], fraction=0.046, pad=0.04)
        cbar.ax.tick_params(labelsize=8)
    
    # 전체 Brixia Score 표시
    total_score = sum(predictions)
    fig.suptitle(
        f'Brixia COVID-19 Severity Score: {total_score}/18\n'
        f'각 부위별 Grad-CAM 히트맵',
        fontsize=16, fontweight='bold'
    )
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✅ 전체 시각화 저장: {save_path}")
    
    plt.close()

def visualize_prediction_summary(predictions, confidences, save_path=None):
    """예측 결과 요약 차트"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 1. 부위별 중증도 막대 그래프
    x = np.arange(6)
    bars = ax1.bar(x, predictions, color=REGION_COLORS, alpha=0.7, edgecolor='black')
    ax1.set_xticks(x)
    ax1.set_xticklabels(REGION_NAMES, rotation=45, ha='right')
    ax1.set_ylabel('중증도 점수', fontsize=12, fontweight='bold')
    ax1.set_ylim(0, 3.5)
    ax1.set_title('부위별 중증도 점수', fontsize=14, fontweight='bold')
    ax1.grid(axis='y', alpha=0.3)
    
    # 각 막대 위에 값 표시
    for i, (bar, conf) in enumerate(zip(bars, confidences)):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{int(height)}\n({conf:.0f}%)',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # 2. 좌우 비교
    left_scores = predictions[:3]  # A, B, C
    right_scores = predictions[3:]  # D, E, F
    
    categories = ['상부', '중부', '하부']
    x2 = np.arange(len(categories))
    width = 0.35
    
    ax2.bar(x2 - width/2, left_scores, width, label='좌폐', 
            color='#FF6B6B', alpha=0.7, edgecolor='black')
    ax2.bar(x2 + width/2, right_scores, width, label='우폐', 
            color='#4ECDC4', alpha=0.7, edgecolor='black')
    
    ax2.set_xticks(x2)
    ax2.set_xticklabels(categories)
    ax2.set_ylabel('중증도 점수', fontsize=12, fontweight='bold')
    ax2.set_ylim(0, 3.5)
    ax2.set_title('좌우 폐 비교', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(axis='y', alpha=0.3)
    
    # 총점 표시
    total_score = sum(predictions)
    fig.suptitle(f'전체 Brixia Score: {total_score}/18', 
                 fontsize=16, fontweight='bold')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✅ 요약 차트 저장: {save_path}")
    
    plt.close()

# ============================================================
# 추론 함수
# ============================================================
# @torch.no_grad()
def predict_single_image(model, image_path, visualize=True):
    """단일 이미지 추론 및 시각화"""
    model.eval()

    with torch.no_grad():
        # 이미지 로드 및 전처리
        img_tensor, original_img = preprocess_image(image_path)
        img_tensor = img_tensor.to(DEVICE)
        
        # 예측
        output, attn_weights = model(img_tensor)  # [1, 6, 4]
        probs = F.softmax(output, dim=-1)  # [1, 6, 4]
        
        predictions = output[0].argmax(dim=-1).cpu().numpy()  # [6]
        confidences = probs[0].max(dim=-1).values.detach().cpu().numpy() * 100  # [6]
    
    # 결과 출력
    print("\n" + "="*60)
    print(f"📊 예측 결과: {os.path.basename(image_path)}")
    print("="*60)
    
    for i, (name, pred, conf) in enumerate(zip(REGION_NAMES, predictions, confidences)):
        print(f"{name}: {CLASS_NAMES[pred]:12s} (신뢰도: {conf:5.1f}%)")
    
    total_score = predictions.sum()
    print(f"\n전체 Brixia Score: {total_score}/18")
    print("="*60)
    
    # 시각화
    if visualize:
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        
        # 1. 전체 6개 부위 Grad-CAM
        all_regions_path = os.path.join(OUTPUT_DIR, f"{base_name}_all_regions.png")
        visualize_all_regions(original_img, model, img_tensor, 
                             predictions, confidences, all_regions_path)
        
        # 2. 예측 요약 차트
        summary_path = os.path.join(OUTPUT_DIR, f"{base_name}_summary.png")
        visualize_prediction_summary(predictions, confidences, summary_path)
        
        # 3. 개별 부위 Grad-CAM (선택적)
        # for idx in range(6):
        #     single_path = os.path.join(OUTPUT_DIR, f"{base_name}_{REGION_NAMES[idx]}.png")
        #     target_layer = model.features[-1]
        #     grad_cam = GradCAM(model, target_layer)
        #     cam, pred_class = grad_cam.generate_cam(img_tensor, region_idx=idx)
        #     visualize_gradcam(original_img, cam, REGION_NAMES[idx], 
        #                      pred_class, confidences[idx], single_path)
    
    return predictions, confidences, attn_weights

# ============================================================
# 메인 실행
# ============================================================
def main():
    print("\n" + "="*70)
    print("🔬 Brixia COVID-19 Grad-CAM 시각화")
    print("="*70)
    
    # 모델 로드
    print(f"\n📦 모델 로딩: {MODEL_PATH}")
    model = EfficientNetB0Classification(
        pretrained=False,
        drop=0.3,
        num_regions=NUM_REGIONS,
        num_classes=NUM_CLASSES
    ).to(DEVICE)
    
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    print("✅ 모델 로드 완료!")
    
    # 테스트 이미지 경로 (예시)
    # 실제 사용 시 경로 수정 필요
    test_image_path = "./data/covid19-xray-severity-scoring/segmented_png/5032497707401895.png"
    
    # 이미지가 존재하는지 확인
    if not os.path.exists(test_image_path):
        print(f"\n⚠️ 테스트 이미지를 찾을 수 없습니다: {test_image_path}")
        print("📝 사용법:")
        print("   1. test_image_path 변수를 실제 이미지 경로로 수정")
        print("   2. 또는 아래 함수를 직접 호출:")
        print("      predict_single_image(model, 'your_image_path.png')")
        return
    
    # 추론 및 시각화
    predictions, confidences, attn_weights = predict_single_image(
        model, test_image_path, visualize=True
    )
    
    print(f"\n✅ 결과가 {OUTPUT_DIR}/ 폴더에 저장되었습니다!")
    print("="*70)

if __name__ == "__main__":
    main()


🔬 Brixia COVID-19 Grad-CAM 시각화

📦 모델 로딩: ./runs_severity_classification/best_efficientnet_b0_classification.pth
✅ 모델 로드 완료!

📊 예측 결과: 5032497707401895.png
A (좌상): 정상 (0)       (신뢰도:  65.3%)
B (좌중): 경증 (1)       (신뢰도:  51.2%)
C (좌하): 중등도 (2)      (신뢰도:  56.5%)
D (우상): 정상 (0)       (신뢰도:  56.0%)
E (우중): 중등도 (2)      (신뢰도:  50.0%)
F (우하): 중증 (3)       (신뢰도:  66.3%)

전체 Brixia Score: 8/18


/tmp/ipykernel_1647408/2806019575.py:254: UserWarning: Glyph 51340 (\N{HANGUL SYLLABLE JWA}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.97])
/tmp/ipykernel_1647408/2806019575.py:254: UserWarning: Glyph 49345 (\N{HANGUL SYLLABLE SANG}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.97])
/tmp/ipykernel_1647408/2806019575.py:254: UserWarning: Glyph 51221 (\N{HANGUL SYLLABLE JEONG}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.97])
/tmp/ipykernel_1647408/2806019575.py:254: UserWarning: Glyph 51473 (\N{HANGUL SYLLABLE JUNG}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.97])
/tmp/ipykernel_1647408/2806019575.py:254: UserWarning: Glyph 44221 (\N{HANGUL SYLLABLE GYEONG}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.97])
/tmp/ipykernel_1647408/2806019575.py:254: UserWarning: Glyph 51613 (\N{HANGUL SYLLABLE JEUNG}) missing from font(s) DejaVu Sans.
  plt.tig

✅ 전체 시각화 저장: ./gradcam_results/5032497707401895_all_regions.png


/tmp/ipykernel_1647408/2806019575.py:309: UserWarning: Glyph 51340 (\N{HANGUL SYLLABLE JWA}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.95])
/tmp/ipykernel_1647408/2806019575.py:309: UserWarning: Glyph 49345 (\N{HANGUL SYLLABLE SANG}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.95])
/tmp/ipykernel_1647408/2806019575.py:309: UserWarning: Glyph 51473 (\N{HANGUL SYLLABLE JUNG}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.95])
/tmp/ipykernel_1647408/2806019575.py:309: UserWarning: Glyph 54616 (\N{HANGUL SYLLABLE HA}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.95])
/tmp/ipykernel_1647408/2806019575.py:309: UserWarning: Glyph 50864 (\N{HANGUL SYLLABLE U}) missing from font(s) DejaVu Sans.
  plt.tight_layout(rect=[0, 0.03, 1, 0.95])
/tmp/ipykernel_1647408/2806019575.py:309: UserWarning: Glyph 51613 (\N{HANGUL SYLLABLE JEUNG}) missing from font(s) DejaVu Sans.
  plt.tight_layou

✅ 요약 차트 저장: ./gradcam_results/5032497707401895_summary.png

✅ 결과가 ./gradcam_results/ 폴더에 저장되었습니다!


In [6]:
import os, random, time
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as TF
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

from tqdm import tqdm
from torch.amp import autocast
from torch.cuda.amp import autocast, GradScaler

# --- 경로 설정 및 하이퍼파라미터 ---
BASE_DIR     = f"./data/covid19-xray-severity-scoring/"
CSV_PATH     = str(Path(BASE_DIR) / "Brixia.csv")
IMAGE_DIR    = str(Path(BASE_DIR) / "segmented_png")

OUT_DIR      = "./runs_severity_classification"
BEST_PATH    = str(Path(OUT_DIR) / "best_efficientnet_b0_classification.pth")
os.makedirs(OUT_DIR, exist_ok=True)

DEVICE       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED         = 42
IMG_SIZE     = 224
BATCH_SIZE   = 32
NUM_CLASSES  = 4   # 0, 1, 2, 3
EPOCHS       = 100  # Single phase training
LR           = 1e-4
WEIGHT_DECAY = 5e-4
AMP          = True
EARLY_STOP_ACC = 0.75  # 🔄 MAE → Accuracy
DROP_RATIO   = 0.3
AUG_RATIO    = 0.5
MIXUP_ALPHA  = 0.2
LABEL_SMOOTHING = 0.1  # ✅ NEW: Label smoothing

# --- 시드 고정 ---
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
set_seed(SEED)

def make_transform_with_label(train: bool, img_size: int = IMG_SIZE, aug_ratio=AUG_RATIO):
    """Brixia Score의 좌우 구조를 고려한 transform"""
    def _tfm(img: Image.Image, label: torch.Tensor = None):
        img = img.convert('RGB')
        img = TF.resize(
            img, 
            [img_size, img_size], 
            interpolation=TF.InterpolationMode.BILINEAR,
            antialias=True
        )
        
        if train:
            # 1. Horizontal Flip (좌우 반전: ABC ↔ DEF)
            if random.random() < aug_ratio:
                img = TF.hflip(img)
                if label is not None:
                    # [A, B, C, D, E, F] → [D, E, F, A, B, C]
                    label = label[[3, 4, 5, 0, 1, 2]]
            
            # 2. 약한 회전 (±5도)
            if random.random() < aug_ratio:
                angle = float(torch.empty(1).uniform_(-5, 5))
                img = TF.rotate(
                    img, 
                    angle, 
                    interpolation=TF.InterpolationMode.BILINEAR,
                    fill=0
                )
            
            # 3. 약한 Translation
            if random.random() < aug_ratio:
                max_dx = 0.05 * img_size
                max_dy = 0.05 * img_size
                translations = (
                    float(torch.empty(1).uniform_(-max_dx, max_dx)),
                    float(torch.empty(1).uniform_(-max_dy, max_dy))
                )
                img = TF.affine(
                    img,
                    angle=0,
                    translate=translations,
                    scale=1.0,
                    shear=0,
                    interpolation=TF.InterpolationMode.BILINEAR,
                    fill=0
                )
            
            # 4. Brightness & Contrast
            if random.random() < aug_ratio:
                brightness_factor = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_brightness(img, brightness_factor)
            
            if random.random() < aug_ratio:
                contrast_factor = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_contrast(img, contrast_factor)
            
            # 5. Gamma Correction
            if random.random() < 0.3:
                gamma = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_gamma(img, gamma)
        
        # Tensor 변환
        img = TF.to_tensor(img)
        
        # Gaussian Noise (train only)
        if train and random.random() < 0.2:
            noise = torch.randn_like(img) * 0.01
            img = img + noise
            img = torch.clamp(img, 0, 1)
        
        # 정규화
        img = TF.normalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        if label is not None:
            return img, label
        return img
    
    return _tfm

def load_and_split_brixia(csv_path, val_ratio=0.2, seed=SEED):
    df = pd.read_csv(csv_path, dtype={'BrixiaScore': str})
    df = df.dropna(subset=['BrixiaScore'])
    df = df[df['BrixiaScore'] != 'nan']
    df = df[df['BrixiaScore'].str.len() == 6].copy()
    
    print(f"유효한 데이터: {len(df)}개")
    
    if 'ConsensusTestset' in df.columns:
        test_df = df[df['ConsensusTestset'] == 1].copy()
        train_val_df = df[df['ConsensusTestset'] == 0].copy()
    else:
        test_df = pd.DataFrame()
        train_val_df = df.copy()
    
    gss = GroupShuffleSplit(n_splits=1, test_size=val_ratio, random_state=seed)
    train_idx, val_idx = next(gss.split(
        train_val_df, 
        groups=train_val_df['StudyId']
    ))
    
    tr_df = train_val_df.iloc[train_idx].copy()
    val_df = train_val_df.iloc[val_idx].copy()
    
    print(f"Train: {len(tr_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
    validate_split(tr_df, val_df, test_df)
    
    return tr_df, val_df, test_df

def validate_split(tr_df, val_df, tt_df):
    train_studies = set(tr_df['StudyId'])
    val_studies = set(val_df['StudyId'])
    test_studies = set(tt_df['StudyId']) if len(tt_df) > 0 else set()
    
    assert len(train_studies & val_studies) == 0, "Train-Val 간 StudyId 중복!"
    assert len(train_studies & test_studies) == 0, "Train-Test 간 StudyId 중복!"
    assert len(val_studies & test_studies) == 0, "Val-Test 간 StudyId 중복!"
    
    for name, data in [('Train', tr_df), ('Val', val_df), ('Test', tt_df)]:
        if len(data) > 0:
            scores = data['BrixiaScore'].apply(lambda x: sum(int(c) for c in x))
            print(f"{name} - Mean: {scores.mean():.2f}, Std: {scores.std():.2f}")
    
    return True

# ============================================================
# Dataset
# ============================================================
class BrixiaDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.img_col = "Filename"
        self.label_col = "BrixiaScore"
        self._validate_data()
    
    def _validate_data(self):
        assert self.img_col in self.df.columns
        assert self.label_col in self.df.columns
        
        invalid_scores = self.df[self.df[self.label_col].str.len() != 6]
        if len(invalid_scores) > 0:
            print(f"⚠️ 경고: {len(invalid_scores)}개의 잘못된 BrixiaScore 발견")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        img_name_from_csv = row[self.img_col]
        img_name = img_name_from_csv.replace('.dcm', '.png')
        img_path = os.path.join(self.img_dir, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"❌ 이미지 로드 오류: {img_path}")
            raise
        
        scores_str = row[self.label_col]
        scores_list = [int(c) for c in scores_str]
        labels = torch.tensor(scores_list, dtype=torch.long)  # 🔄 long으로 변경
        
        if self.transform:
            image, labels = self.transform(image, labels)
        else:
            image = TF.to_tensor(image)
            image = TF.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, labels

def create_dataloaders(tr_df, val_df, tt_df, img_dir, 
                       batch_size=32, img_size=224, num_workers=4):
    train_transform = make_transform_with_label(train=True, img_size=img_size)
    val_transform = make_transform_with_label(train=False, img_size=img_size)
    
    tr_ds = BrixiaDataset(tr_df, img_dir, transform=train_transform)
    val_ds = BrixiaDataset(val_df, img_dir, transform=val_transform)
    tt_ds = BrixiaDataset(tt_df, img_dir, transform=val_transform)
    
    tr_loader = DataLoader(
        tr_ds, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available(),
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_ds, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )
    
    tt_loader = DataLoader(
        tt_ds, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )
    
    print("✅ DataLoader 준비 완료")
    print(f"   Train: {len(tr_ds)} samples, {len(tr_loader)} batches")
    print(f"   Val:   {len(val_ds)} samples, {len(val_loader)} batches")
    print(f"   Test:  {len(tt_ds)} samples, {len(tt_loader)} batches")
    
    return tr_loader, val_loader, tt_loader

# ============================================================
# Loss Function - Ordinal Classification
# ============================================================
def calculate_class_weights(labels, num_classes=4, method='sqrt_inverse'):
    """클래스 불균형 해결을 위한 가중치 계산"""
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    labels_flat = labels.flatten()
    counts = np.bincount(labels_flat.astype(int), minlength=num_classes)
    
    if method == 'sqrt_inverse':
        weights = 1.0 / (np.sqrt(counts) + 1e-6)
    elif method == 'inverse':
        weights = 1.0 / (counts + 1e-6)
    else:
        total = len(labels_flat)
        weights = total / (num_classes * (counts + 1e-6))
    
    weights = weights / weights.mean()
    return torch.FloatTensor(weights)

def print_class_distribution(labels):
    """클래스 분포 시각화"""
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    print("=" * 60)
    print("Class Distribution Analysis")
    print("=" * 60)
    
    region_names = ['A', 'B', 'C', 'D', 'E', 'F']
    for idx, name in enumerate(region_names):
        region_labels = labels[:, idx]
        counts = np.bincount(region_labels.astype(int), minlength=4)
        total = counts.sum()
        
        print(f"\n{name}:")
        for cls in range(4):
            pct = 100 * counts[cls] / total if total > 0 else 0
            bar = '█' * int(pct / 2)
            print(f"  Class {cls}: {counts[cls]:4d} ({pct:5.1f}%) {bar}")
    
    print("=" * 60)

class OrdinalRegressionLoss(nn.Module):
    """
    ✅ Ordinal Regression Loss (순서형 회귀)
    0 < 1 < 2 < 3의 순서를 명시적으로 학습
    """
    def __init__(self, num_classes=4):
        super().__init__()
        self.num_classes = num_classes
        
    def forward(self, logits, target):
        """
        logits: [B*6, num_classes-1] - cumulative logits
        target: [B*6] - class labels (0~3)
        """
        # Cumulative labels 생성
        # Class 0: [0, 0, 0]
        # Class 1: [1, 0, 0]
        # Class 2: [1, 1, 0]
        # Class 3: [1, 1, 1]
        batch_size = target.size(0)
        target_expanded = target.unsqueeze(1)  # [B*6, 1]
        
        # [0, 1, 2, ..., num_classes-2]
        thresholds = torch.arange(self.num_classes - 1).to(target.device)
        thresholds = thresholds.unsqueeze(0).expand(batch_size, -1)  # [B*6, 3]
        
        # target > threshold이면 1, 아니면 0
        cumulative_target = (target_expanded > thresholds).float()  # [B*6, 3]
        
        # Binary cross entropy for each threshold
        loss = F.binary_cross_entropy_with_logits(
            logits, cumulative_target, reduction='none'
        )
        
        return loss.mean()

class AdaptiveOrdinalLoss(nn.Module):
    """
    ✅ Ordinal Loss + Class Weights + Part Weights
    순서를 고려하면서 불균형도 해결
    """
    
    def __init__(self, train_labels, num_classes=4, use_class_weights=True, 
                 part_weights=None, ordinal_weight=0.5):
        super().__init__()
        self.num_classes = num_classes
        self.ordinal_weight = ordinal_weight  # Ordinal loss와 CE loss의 비율
        
        if isinstance(train_labels, torch.Tensor):
            train_labels_np = train_labels.cpu().numpy()
        else:
            train_labels_np = train_labels
        
        print_class_distribution(train_labels_np)
        
        # 클래스 가중치
        if use_class_weights:
            class_weights = calculate_class_weights(train_labels_np, num_classes=num_classes)
            self.register_buffer('class_weights', class_weights)
            print(f"✅ Class weights: {class_weights.numpy()}")
        else:
            self.class_weights = None
        
        # 부위별 가중치
        if part_weights is None:
            self.part_weights = torch.ones(6)
        else:
            self.part_weights = torch.tensor(part_weights, dtype=torch.float32)
        self.register_buffer('part_weights_buf', self.part_weights)
        print(f"✅ Part weights: {self.part_weights.numpy()}")
        print(f"✅ Ordinal weight: {ordinal_weight}")
        
        # Ordinal loss
        self.ordinal_loss = OrdinalRegressionLoss(num_classes)

    def forward(self, pred_dict, target, use_mixup=False):
        """
        pred_dict: {'logits': [B, 6, 4], 'ordinal_logits': [B, 6, 3]}
        target: [B, 6]
        """
        pred_logits = pred_dict['logits']  # [B, 6, 4]
        ordinal_logits = pred_dict['ordinal_logits']  # [B, 6, 3]
        
        B, num_regions, num_classes = pred_logits.shape
        
        # Reshape
        pred_logits_flat = pred_logits.view(B * num_regions, num_classes)
        ordinal_logits_flat = ordinal_logits.view(B * num_regions, num_classes - 1)
        target_flat = target.view(B * num_regions)
        
        # 1. CrossEntropyLoss (기본 분류)
        if self.class_weights is not None and not use_mixup:
            ce_loss = F.cross_entropy(
                pred_logits_flat, 
                target_flat,
                weight=self.class_weights.to(pred_logits.device),
                reduction='none'
            )
        else:
            ce_loss = F.cross_entropy(
                pred_logits_flat, 
                target_flat,
                reduction='none'
            )
        
        # 2. Ordinal Loss (순서 학습)
        ord_loss = self.ordinal_loss(ordinal_logits_flat, target_flat)
        
        # 3. 결합
        total_loss_flat = (1 - self.ordinal_weight) * ce_loss + self.ordinal_weight * ord_loss
        total_loss = total_loss_flat.view(B, num_regions)
        
        # 부위별 가중치 적용
        part_weights = self.part_weights_buf.to(pred_logits.device)
        total_loss = total_loss * part_weights.unsqueeze(0)
        
        part_losses = total_loss.mean(dim=0)
        
        return total_loss.mean(), part_losses

# ============================================================
# Mixup (분류용)
# ============================================================
def mixup_data_classification(x, y, alpha=MIXUP_ALPHA):
    """
    ✅ NEW: 분류용 Mixup
    y는 one-hot으로 변환 후 mixup
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    
    # One-hot encoding
    y_onehot = F.one_hot(y, num_classes=NUM_CLASSES).float()  # [B, 6, 4]
    y_onehot_shuffled = y_onehot[index]
    
    mixed_y = lam * y_onehot + (1 - lam) * y_onehot_shuffled  # [B, 6, 4]
    
    return mixed_x, mixed_y, lam

def mixup_criterion(criterion, pred_dict, y_mixed, lam):
    """Mixup을 위한 손실 계산 (Ordinal Loss 지원)"""
    pred_logits = pred_dict['logits']
    ordinal_logits = pred_dict['ordinal_logits']
    
    B, num_regions, num_classes = pred_logits.shape
    
    # Reshape
    pred_flat = pred_logits.view(B * num_regions, num_classes)
    ordinal_flat = ordinal_logits.view(B * num_regions, num_classes - 1)
    target_flat = y_mixed.view(B * num_regions, num_classes)
    
    # Soft target loss (CE part)
    log_probs = F.log_softmax(pred_flat, dim=1)
    ce_loss = -(target_flat * log_probs).sum(dim=1)
    
    # Ordinal part는 mixup에서 skip (hard label만 사용)
    loss = ce_loss  # Mixup 시에는 ordinal loss 제외
    loss = loss.view(B, num_regions)
    
    # 부위별 가중치 적용
    if hasattr(criterion, 'part_weights_buf'):
        part_weights = criterion.part_weights_buf.to(pred_logits.device)
        loss = loss * part_weights.unsqueeze(0)
    
    part_losses = loss.mean(dim=0)
    return loss.mean(), part_losses

# ============================================================
# Metrics (분류용)
# ============================================================
@torch.no_grad()
def calculate_classification_metrics(pred_dict, labels):
    """
    ✅ 분류 지표 계산
    pred_dict: {'logits': [B, 6, 4], 'ordinal_logits': [B, 6, 3]}
    labels: [B, 6]
    """
    pred_logits = pred_dict['logits']
    
    # 예측 클래스
    preds = pred_logits.argmax(dim=-1)  # [B, 6]
    
    # Exact match accuracy
    exact_acc = (preds == labels).float().mean().item()
    
    # Off-by-1 accuracy (인접 클래스 허용)
    off_by_1 = (torch.abs(preds - labels) <= 1).float().mean().item()
    
    # Per-region accuracy
    region_acc = (preds == labels).float().mean(dim=0)  # [6]
    
    # MAE (참고용)
    mae = torch.abs(preds.float() - labels.float()).mean().item()
    
    return exact_acc, off_by_1, mae, region_acc

# ============================================================
# Model (분류용으로 변경)
# ============================================================
class EfficientNetB0Classification(nn.Module):
    """
    ✅ Ordinal Classification 모델
    - 각 부위(A~F)마다 4개 클래스(0~3) 예측
    - Self-Attention으로 부위 간 상관관계 학습
    - Ordinal Regression Head 추가 (순서 학습)
    """
    def __init__(self, pretrained=True, drop=0.3, num_regions=6, num_classes=4):
        super().__init__()
        
        weights = EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None
        backbone = efficientnet_b0(weights=weights)
        
        self.features = backbone.features
        in_feat = 1280
        
        # Positional embedding
        self.pos_embed = nn.Parameter(torch.randn(1, 49, in_feat))
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        
        # Region queries
        self.region_queries = nn.Parameter(torch.randn(num_regions, in_feat))
        nn.init.xavier_uniform_(self.region_queries)
        
        # Cross attention (이미지 특징 → 부위별 특징)
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=in_feat,
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )
        
        self.norm1 = nn.LayerNorm(in_feat)
        
        # Self-Attention (부위 간 상관관계 학습)
        self.self_attention = nn.MultiheadAttention(
            embed_dim=in_feat,
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )
        
        self.norm2 = nn.LayerNorm(in_feat)
        self.norm3 = nn.LayerNorm(in_feat)
        
        # FFN
        self.ffn = nn.Sequential(
            nn.Linear(in_feat, in_feat * 2),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(in_feat * 2, in_feat),
            nn.Dropout(drop)
        )
        
        self.norm4 = nn.LayerNorm(in_feat)
        
        # Shared feature extractor
        self.shared_fc = nn.Sequential(
            nn.Linear(in_feat, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(drop)
        )
        
        # ✅ Classification heads (일반 분류)
        self.classification_heads = nn.ModuleList([
            nn.Linear(128, num_classes) for _ in range(num_regions)
        ])
        
        # ✅ Ordinal Regression heads (순서형 회귀)
        # num_classes-1 개의 threshold를 학습
        self.ordinal_heads = nn.ModuleList([
            nn.Linear(128, num_classes - 1) for _ in range(num_regions)
        ])
    
    def forward(self, x):
        B = x.size(0)
        
        # Feature extraction
        feat = self.features(x)  # [B, 1280, 7, 7]
        feat = feat.flatten(2).transpose(1, 2)  # [B, 49, 1280]
        feat = feat + self.pos_embed
        
        # Region queries
        queries = self.region_queries.unsqueeze(0).expand(B, -1, -1)  # [B, 6, 1280]
        
        # Cross attention (이미지 → 부위)
        attn_out, _ = self.cross_attention(
            query=queries,
            key=feat,
            value=feat
        )
        attn_out = self.norm1(attn_out + queries)  # [B, 6, 1280]
        
        # Self-Attention (부위 간 상관관계)
        self_attn_out, attn_weights = self.self_attention(
            query=attn_out,
            key=attn_out,
            value=attn_out
        )
        attn_out = self.norm2(attn_out + self_attn_out)  # [B, 6, 1280]
        
        # FFN
        ffn_out = self.ffn(attn_out)
        attn_out = self.norm3(attn_out + ffn_out)  # [B, 6, 1280]
        
        # Shared feature extraction
        shared_features = self.shared_fc(attn_out)  # [B, 6, 128]
        
        # ✅ 각 부위별 예측 (Classification + Ordinal)
        classification_outputs = []
        ordinal_outputs = []
        
        for i in range(len(self.classification_heads)):
            region_feat = shared_features[:, i, :]  # [B, 128]
            
            # Classification logits
            class_logits = self.classification_heads[i](region_feat)  # [B, 4]
            classification_outputs.append(class_logits)
            
            # Ordinal logits (cumulative)
            ordinal_logits = self.ordinal_heads[i](region_feat)  # [B, 3]
            ordinal_outputs.append(ordinal_logits)
        
        class_out = torch.stack(classification_outputs, dim=1)  # [B, 6, 4]
        ordinal_out = torch.stack(ordinal_outputs, dim=1)  # [B, 6, 3]
        
        return {
            'logits': class_out,
            'ordinal_logits': ordinal_out,
            'attn_weights': attn_weights
        }

# ============================================================
# Training Functions (분류용 수정)
# ============================================================
def train_epoch(model, tr_loader, criterion, optimizer, scaler, device, 
                amp=True, use_mixup=True):
    model.train()
    run_loss = run_acc = run_off1 = run_mae = n = 0
    part_losses_sum = torch.zeros(6)
    
    pbar = tqdm(tr_loader, desc="Train", leave=False)
    
    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)  # [B, 6]
        
        # Mixup 적용
        is_mixup = use_mixup and (random.random() < 0.5)
        
        optimizer.zero_grad(set_to_none=True)
        
        with autocast(enabled=amp):
            if is_mixup:
                imgs_mixed, labels_mixed, lam = mixup_data_classification(imgs, labels)
                pred_dict = model(imgs_mixed)
                loss, part_losses = mixup_criterion(criterion, pred_dict, labels_mixed, lam)
            else:
                pred_dict = model(imgs)
                loss, part_losses = criterion(pred_dict, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Metrics (원본 labels로 계산)
        exact_acc, off_by_1, mae, _ = calculate_classification_metrics(pred_dict, labels)
        
        bs = imgs.size(0)
        run_loss += loss.item() * bs
        run_acc += exact_acc * bs
        run_off1 += off_by_1 * bs
        run_mae += mae * bs
        part_losses_sum += part_losses.cpu() * bs
        n += bs
        
        pbar.set_postfix(
            loss=f"{run_loss/n:.4f}",
            acc=f"{run_acc/n:.4f}",
            mae=f"{run_mae/n:.4f}"
        )
    
    part_losses_avg = part_losses_sum / n
    
    return run_loss/n, run_acc/n, run_off1/n, run_mae/n, part_losses_avg

@torch.no_grad()
def evaluate(model, val_loader, criterion, device, split='val'):
    model.eval()
    run_loss = run_acc = run_off1 = run_mae = n = 0
    part_losses_sum = torch.zeros(6)
    region_acc_sum = torch.zeros(6)
    
    pbar = tqdm(val_loader, desc=f"{split.capitalize()}", leave=False)
    
    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        pred_dict = model(imgs)
        loss, part_losses = criterion(pred_dict, labels)
        
        exact_acc, off_by_1, mae, region_acc = calculate_classification_metrics(pred_dict, labels)
        
        bs = imgs.size(0)
        run_loss += loss.item() * bs
        run_acc += exact_acc * bs
        run_off1 += off_by_1 * bs
        run_mae += mae * bs
        part_losses_sum += part_losses.cpu() * bs
        region_acc_sum += region_acc.cpu() * bs
        n += bs
        
        pbar.set_postfix(
            loss=f"{run_loss/n:.4f}",
            acc=f"{run_acc/n:.4f}"
        )
    
    avg_loss = run_loss/n
    avg_acc = run_acc/n
    avg_off1 = run_off1/n
    avg_mae = run_mae/n
    part_losses_avg = part_losses_sum / n
    region_acc_avg = region_acc_sum / n
    
    print(f"[{split}] loss:{avg_loss:.4f} acc:{avg_acc:.4f} "
          f"off1:{avg_off1:.4f} mae:{avg_mae:.4f}")
    print(f"  Region Acc: {region_acc_avg.numpy().round(3)}")
    
    return avg_loss, avg_acc, avg_off1, avg_mae, part_losses_avg, region_acc_avg

def get_lrs(optimizer):
    return [pg['lr'] for pg in optimizer.param_groups]

# ============================================================
# Main Function
# ============================================================
def main():
    print("\n" + "="*70)
    print("🚀 Brixia COVID-19 Ordinal Classification Training")
    print("   💡 80% 목표를 위한 핵심 기술:")
    print("   ✅ Ordinal Loss - 순서형 회귀 (0<1<2<3)")
    print("   ✅ 클래스 불균형 처리 (자동 가중치)")
    print("   ✅ 부위 간 상관관계 학습 (Self-Attention)")
    print("   ✅ Mixed Precision Training (AMP)")
    print("   ✅ Mixup Augmentation")
    print("="*70)
    
    # 데이터 준비
    print("\n📂 Loading data...")
    tr_df, val_df, tt_df = load_and_split_brixia(CSV_PATH)
    
    print("\n📦 Creating DataLoaders...")
    tr_loader, val_loader, tt_loader = create_dataloaders(
        tr_df, val_df, tt_df, img_dir=IMAGE_DIR, 
        batch_size=BATCH_SIZE, img_size=IMG_SIZE, num_workers=4
    )
    
    # Train labels 추출
    train_labels = torch.cat([labels for _, labels in tr_loader], dim=0)
    
    # ========================================
    # 학습 준비
    # ========================================
    print("\n" + "="*70)
    print("📍 Training Setup")
    print("="*70)
    
    # ✅ Ordinal Loss + 클래스 가중치
    criterion = AdaptiveOrdinalLoss(
        train_labels, 
        num_classes=NUM_CLASSES,
        use_class_weights=True,
        part_weights=None,
        ordinal_weight=0.5  # CE:Ordinal = 50:50
    )
    
    # ✅ 부위 간 상관관계를 고려하는 모델
    model = EfficientNetB0Classification(
        pretrained=True, 
        drop=DROP_RATIO,
        num_regions=6,
        num_classes=NUM_CLASSES
    ).to(DEVICE)
    print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, min_lr=1e-6
    )
    scaler = GradScaler(enabled=AMP)
    
    best_acc = 0.0
    best_mae = float('inf')
    patience_counter = 0
    max_patience = 15
    
    # ========================================
    # 학습 루프
    # ========================================
    print("\n" + "="*70)
    print("🏋️ Training Start")
    print("="*70)
    
    for ep in range(1, EPOCHS + 1):
        t0 = time.time()
        
        tr_loss, tr_acc, tr_off1, tr_mae, tr_part_losses = train_epoch(
            model, tr_loader, criterion, optimizer, scaler, DEVICE, AMP
        )
        
        val_loss, val_acc, val_off1, val_mae, val_part_losses, val_region_acc = evaluate(
            model, val_loader, criterion, DEVICE, split='val'
        )
        
        scheduler.step(val_acc)
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            best_mae = val_mae
            patience_counter = 0
            torch.save({
                'epoch': ep,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': best_acc,
                'val_mae': best_mae,
                'val_off1': val_off1,
                'region_acc': val_region_acc,
            }, BEST_PATH)
            print(f"✅ New Best! (Acc={best_acc:.4f}, MAE={best_mae:.4f})")
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= max_patience:
            print(f"\n⏹️ Early stopping at epoch {ep}")
            break
        
        elapsed = time.time() - t0
        print(f"\n[Epoch {ep:03d}/{EPOCHS}]")
        print(f"  Train - loss:{tr_loss:.4f} acc:{tr_acc:.4f} off1:{tr_off1:.4f} mae:{tr_mae:.4f}")
        print(f"  Val   - loss:{val_loss:.4f} acc:{val_acc:.4f} off1:{val_off1:.4f} mae:{val_mae:.4f}")
        print(f"  Part losses (Val): {val_part_losses.numpy().round(3)}")
        print(f"  Region Acc (Val): {val_region_acc.numpy().round(3)}")
        print(f"  LR:{get_lrs(optimizer)[0]:.2e} | {elapsed:.1f}s | Patience:{patience_counter}/{max_patience}")
        print("-" * 70)
    
    # ========================================
    # Test Evaluation
    # ========================================
    print("\n" + "="*70)
    print("🎉 Training Finished!")
    print("="*70)
    
    if len(tt_loader) > 0:
        print("\n📊 Test evaluation with best model...")
        checkpoint = torch.load(BEST_PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        
        tt_loss, tt_acc, tt_off1, tt_mae, tt_part_losses, tt_region_acc = evaluate(
            model, tt_loader, criterion, DEVICE, split='test'
        )
        
        print(f"\n🏆 Test Results:")
        print(f"   Accuracy: {tt_acc:.4f}")
        print(f"   Off-by-1: {tt_off1:.4f}")
        print(f"   MAE: {tt_mae:.4f}")
        print(f"   Region Acc: {tt_region_acc.numpy().round(3)}")
        print(f"   Part losses: {tt_part_losses.numpy().round(3)}")
    
    print(f"\n💾 Best model saved: {BEST_PATH}")
    print(f"📈 Best Validation Accuracy: {best_acc:.4f}")
    print(f"📉 Best Validation MAE: {best_mae:.4f}")
    
    # 성능 향상 제안
    print("\n" + "="*70)
    print("💡 추가 성능 향상을 위한 제안:")
    print("="*70)
    print("1. 🎯 앙상블: 다른 시드로 3~5개 모델 학습 후 투표")
    print("2. 🔬 의료 전용 pretrained model 사용:")
    print("   - CheXpert, MIMIC-CXR 등으로 사전학습된 모델")
    print("3. 📊 데이터 추가:")
    print("   - 외부 COVID-19 데이터셋 활용")
    print("   - Pseudo-labeling으로 unlabeled 데이터 활용")
    print("4. 🎨 고급 증강:")
    print("   - CutMix, AugMix, RandAugment")
    print("   - Test-Time Augmentation (TTA)")
    print("5. 🧪 하이퍼파라미터 튜닝:")
    print("   - ordinal_weight 조정 (0.3~0.7)")
    print("   - Learning rate, batch size 실험")
    print("="*70)
    
    print("\n✅ 이제 gradcam_inference.py를 실행하여 결과를 시각화하세요!")
    print("="*70)

if __name__ == "__main__":
    main()


🚀 Brixia COVID-19 Ordinal Classification Training
   💡 80% 목표를 위한 핵심 기술:
   ✅ Ordinal Loss - 순서형 회귀 (0<1<2<3)
   ✅ 클래스 불균형 처리 (자동 가중치)
   ✅ 부위 간 상관관계 학습 (Self-Attention)
   ✅ Mixed Precision Training (AMP)
   ✅ Mixup Augmentation

📂 Loading data...
유효한 데이터: 4695개
Train: 3637, Val: 912, Test: 146
Train - Mean: 8.31, Std: 4.26
Val - Mean: 8.35, Std: 4.15
Test - Mean: 7.78, Std: 4.20

📦 Creating DataLoaders...
✅ DataLoader 준비 완료
   Train: 3637 samples, 113 batches
   Val:   912 samples, 29 batches
   Test:  146 samples, 5 batches

📍 Training Setup
Class Distribution Analysis

A:
  Class 0: 1810 ( 50.1%) █████████████████████████
  Class 1: 1126 ( 31.1%) ███████████████
  Class 2:  446 ( 12.3%) ██████
  Class 3:  234 (  6.5%) ███

B:
  Class 0:  721 ( 19.9%) █████████
  Class 1:  949 ( 26.2%) █████████████
  Class 2: 1171 ( 32.4%) ████████████████
  Class 3:  775 ( 21.4%) ██████████

C:
  Class 0:  406 ( 11.2%) █████
  Class 1:  783 ( 21.7%) ██████████
  Class 2: 1300 ( 36.0%) ███████████

[val] loss:0.8177 acc:0.5000 off1:0.8849 mae:0.6274
  Region Acc: [0.535 0.478 0.442 0.604 0.477 0.464]
✅ New Best! (Acc=0.5000, MAE=0.6274)

[Epoch 001/100]
  Train - loss:1.0706 acc:0.4070 off1:0.8209 mae:0.8038
  Val   - loss:0.8177 acc:0.5000 off1:0.8849 mae:0.6274
  Part losses (Val): [0.805 0.849 0.851 0.726 0.832 0.843]
  Region Acc (Val): [0.535 0.478 0.442 0.604 0.477 0.464]
  LR:1.00e-04 | 17.4s | Patience:0/15
----------------------------------------------------------------------


[val] loss:0.7633 acc:0.5055 off1:0.8977 mae:0.6080
  Region Acc: [0.553 0.469 0.433 0.623 0.48  0.475]
✅ New Best! (Acc=0.5055, MAE=0.6080)

[Epoch 002/100]
  Train - loss:0.9917 acc:0.4487 off1:0.8573 mae:0.7199
  Val   - loss:0.7633 acc:0.5055 off1:0.8977 mae:0.6080
  Part losses (Val): [0.741 0.79  0.816 0.671 0.777 0.785]
  Region Acc (Val): [0.553 0.469 0.433 0.623 0.48  0.475]
  LR:1.00e-04 | 13.4s | Patience:0/15
----------------------------------------------------------------------


[val] loss:0.7017 acc:0.5481 off1:0.9262 mae:0.5305
  Region Acc: [0.562 0.536 0.515 0.657 0.523 0.495]
✅ New Best! (Acc=0.5481, MAE=0.5305)

[Epoch 003/100]
  Train - loss:0.9166 acc:0.4787 off1:0.8809 mae:0.6574
  Val   - loss:0.7017 acc:0.5481 off1:0.9262 mae:0.5305
  Part losses (Val): [0.684 0.709 0.732 0.608 0.727 0.75 ]
  Region Acc (Val): [0.562 0.536 0.515 0.657 0.523 0.495]
  LR:1.00e-04 | 14.2s | Patience:0/15
----------------------------------------------------------------------


[val] loss:0.6826 acc:0.5590 off1:0.9315 mae:0.5148
  Region Acc: [0.586 0.539 0.524 0.649 0.548 0.508]
✅ New Best! (Acc=0.5590, MAE=0.5148)

[Epoch 004/100]
  Train - loss:0.8872 acc:0.4768 off1:0.8747 mae:0.6703
  Val   - loss:0.6826 acc:0.5590 off1:0.9315 mae:0.5148
  Part losses (Val): [0.677 0.703 0.711 0.587 0.697 0.72 ]
  Region Acc (Val): [0.586 0.539 0.524 0.649 0.548 0.508]
  LR:1.00e-04 | 14.2s | Patience:0/15
----------------------------------------------------------------------


...

[val] loss:0.6555 acc:0.5828 off1:0.9536 mae:0.4682
  Region Acc: [0.598 0.577 0.557 0.669 0.578 0.519]

[Epoch 025/100]
  Train - loss:0.6583 acc:0.5978 off1:0.9216 mae:0.4983
  Val   - loss:0.6555 acc:0.5828 off1:0.9536 mae:0.4682
  Part losses (Val): [0.659 0.658 0.684 0.569 0.647 0.717]
  Region Acc (Val): [0.598 0.577 0.557 0.669 0.578 0.519]
  LR:2.50e-05 | 13.2s | Patience:12/15
----------------------------------------------------------------------


[val] loss:0.6572 acc:0.5828 off1:0.9583 mae:0.4620
  Region Acc: [0.606 0.576 0.541 0.664 0.582 0.527]

[Epoch 026/100]
  Train - loss:0.6394 acc:0.6116 off1:0.9248 mae:0.4810
  Val   - loss:0.6572 acc:0.5828 off1:0.9583 mae:0.4620
  Part losses (Val): [0.647 0.659 0.692 0.571 0.659 0.715]
  Region Acc (Val): [0.606 0.576 0.541 0.664 0.582 0.527]
  LR:2.50e-05 | 13.1s | Patience:13/15
----------------------------------------------------------------------


[val] loss:0.6470 acc:0.5846 off1:0.9572 mae:0.4614
  Region Acc: [0.613 0.567 0.567 0.662 0.57  0.529]

[Epoch 027/100]
  Train - loss:0.6684 acc:0.5970 off1:0.9136 mae:0.5092
  Val   - loss:0.6470 acc:0.5846 off1:0.9572 mae:0.4614
  Part losses (Val): [0.633 0.653 0.68  0.56  0.646 0.709]
  Region Acc (Val): [0.613 0.567 0.567 0.662 0.57  0.529]
  LR:2.50e-05 | 13.8s | Patience:14/15
----------------------------------------------------------------------


[val] loss:0.6601 acc:0.5793 off1:0.9556 mae:0.4682
  Region Acc: [0.599 0.561 0.559 0.669 0.57  0.518]

⏹️ Early stopping at epoch 28

🎉 Training Finished!

📊 Test evaluation with best model...


[test] loss:0.5895 acc:0.5982 off1:0.9703 mae:0.4326
  Region Acc: [0.664 0.555 0.603 0.719 0.521 0.527]

🏆 Test Results:
   Accuracy: 0.5982
   Off-by-1: 0.9703
   MAE: 0.4326
   Region Acc: [0.664 0.555 0.603 0.719 0.521 0.527]
   Part losses: [0.53  0.62  0.607 0.503 0.614 0.663]

💾 Best model saved: runs_severity_classification/best_efficientnet_b0_classification.pth
📈 Best Validation Accuracy: 0.5963
📉 Best Validation MAE: 0.4527

💡 추가 성능 향상을 위한 제안:
1. 🎯 앙상블: 다른 시드로 3~5개 모델 학습 후 투표
2. 🔬 의료 전용 pretrained model 사용:
   - CheXpert, MIMIC-CXR 등으로 사전학습된 모델
3. 📊 데이터 추가:
   - 외부 COVID-19 데이터셋 활용
   - Pseudo-labeling으로 unlabeled 데이터 활용
4. 🎨 고급 증강:
   - CutMix, AugMix, RandAugment
   - Test-Time Augmentation (TTA)
5. 🧪 하이퍼파라미터 튜닝:
   - ordinal_weight 조정 (0.3~0.7)
   - Learning rate, batch size 실험

✅ 이제 gradcam_inference.py를 실행하여 결과를 시각화하세요!


In [13]:
import os, random, time
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as TF
import torchxrayvision as xrv  # ✅ 의료 영상 전용 라이브러리

from tqdm import tqdm
from torch.amp import autocast
from torch.cuda.amp import autocast, GradScaler

# --- 경로 설정 및 하이퍼파라미터 ---
BASE_DIR     = f"./data/covid19-xray-severity-scoring/"
CSV_PATH     = str(Path(BASE_DIR) / "Brixia.csv")
IMAGE_DIR    = str(Path(BASE_DIR) / "segmented_png")

OUT_DIR      = "./runs_severity_classification"
BEST_PATH    = str(Path(OUT_DIR) / "best_densenet121_mimic_classification.pth")
os.makedirs(OUT_DIR, exist_ok=True)

DEVICE       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED         = 42
IMG_SIZE     = 224
BATCH_SIZE   = 256
NUM_CLASSES  = 4   # 0, 1, 2, 3
EPOCHS       = 100  # Single phase training
LR           = 1e-4
WEIGHT_DECAY = 5e-4
AMP          = True
EARLY_STOP_ACC = 0.75  # 🔄 MAE → Accuracy
DROP_RATIO   = 0.3
AUG_RATIO    = 0.5
MIXUP_ALPHA  = 0.2
LABEL_SMOOTHING = 0.1  # ✅ NEW: Label smoothing

# --- 시드 고정 ---
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
set_seed(SEED)

def make_transform_with_label(train: bool, img_size: int = IMG_SIZE, aug_ratio=AUG_RATIO):
    """Brixia Score의 좌우 구조를 고려한 transform"""
    def _tfm(img: Image.Image, label: torch.Tensor = None):
        img = img.convert('RGB')
        img = TF.resize(
            img, 
            [img_size, img_size], 
            interpolation=TF.InterpolationMode.BILINEAR,
            antialias=True
        )
        
        if train:
            # 1. Horizontal Flip (좌우 반전: ABC ↔ DEF)
            if random.random() < aug_ratio:
                img = TF.hflip(img)
                if label is not None:
                    # [A, B, C, D, E, F] → [D, E, F, A, B, C]
                    label = label[[3, 4, 5, 0, 1, 2]]
            
            # 2. 약한 회전 (±5도)
            if random.random() < aug_ratio:
                angle = float(torch.empty(1).uniform_(-5, 5))
                img = TF.rotate(
                    img, 
                    angle, 
                    interpolation=TF.InterpolationMode.BILINEAR,
                    fill=0
                )
            
            # 3. 약한 Translation
            if random.random() < aug_ratio:
                max_dx = 0.05 * img_size
                max_dy = 0.05 * img_size
                translations = (
                    float(torch.empty(1).uniform_(-max_dx, max_dx)),
                    float(torch.empty(1).uniform_(-max_dy, max_dy))
                )
                img = TF.affine(
                    img,
                    angle=0,
                    translate=translations,
                    scale=1.0,
                    shear=0,
                    interpolation=TF.InterpolationMode.BILINEAR,
                    fill=0
                )
            
            # 4. Brightness & Contrast
            if random.random() < aug_ratio:
                brightness_factor = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_brightness(img, brightness_factor)
            
            if random.random() < aug_ratio:
                contrast_factor = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_contrast(img, contrast_factor)
            
            # 5. Gamma Correction
            if random.random() < 0.3:
                gamma = float(torch.empty(1).uniform_(0.9, 1.1))
                img = TF.adjust_gamma(img, gamma)
        
        # Tensor 변환
        img = TF.to_tensor(img)
        
        # Gaussian Noise (train only)
        if train and random.random() < 0.2:
            noise = torch.randn_like(img) * 0.01
            img = img + noise
            img = torch.clamp(img, 0, 1)
        
        # 정규화 (torchxrayvision 표준)
        # MIMIC-CXR 데이터로 학습된 모델은 [-1024, 1024] 범위를 사용
        # 하지만 일반 RGB 이미지는 ImageNet 정규화 유지
        img = TF.normalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        if label is not None:
            return img, label
        return img
    
    return _tfm

def load_and_split_brixia(csv_path, val_ratio=0.2, seed=SEED):
    df = pd.read_csv(csv_path, dtype={'BrixiaScore': str})
    df = df.dropna(subset=['BrixiaScore'])
    df = df[df['BrixiaScore'] != 'nan']
    df = df[df['BrixiaScore'].str.len() == 6].copy()
    
    print(f"유효한 데이터: {len(df)}개")
    
    if 'ConsensusTestset' in df.columns:
        test_df = df[df['ConsensusTestset'] == 1].copy()
        train_val_df = df[df['ConsensusTestset'] == 0].copy()
    else:
        test_df = pd.DataFrame()
        train_val_df = df.copy()
    
    gss = GroupShuffleSplit(n_splits=1, test_size=val_ratio, random_state=seed)
    train_idx, val_idx = next(gss.split(
        train_val_df, 
        groups=train_val_df['StudyId']
    ))
    
    tr_df = train_val_df.iloc[train_idx].copy()
    val_df = train_val_df.iloc[val_idx].copy()
    
    print(f"Train: {len(tr_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
    validate_split(tr_df, val_df, test_df)
    
    return tr_df, val_df, test_df

def validate_split(tr_df, val_df, tt_df):
    train_studies = set(tr_df['StudyId'])
    val_studies = set(val_df['StudyId'])
    test_studies = set(tt_df['StudyId']) if len(tt_df) > 0 else set()
    
    assert len(train_studies & val_studies) == 0, "Train-Val 간 StudyId 중복!"
    assert len(train_studies & test_studies) == 0, "Train-Test 간 StudyId 중복!"
    assert len(val_studies & test_studies) == 0, "Val-Test 간 StudyId 중복!"
    
    for name, data in [('Train', tr_df), ('Val', val_df), ('Test', tt_df)]:
        if len(data) > 0:
            scores = data['BrixiaScore'].apply(lambda x: sum(int(c) for c in x))
            print(f"{name} - Mean: {scores.mean():.2f}, Std: {scores.std():.2f}")
    
    return True

# ============================================================
# Dataset
# ============================================================
class BrixiaDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.img_col = "Filename"
        self.label_col = "BrixiaScore"
        self._validate_data()
    
    def _validate_data(self):
        assert self.img_col in self.df.columns
        assert self.label_col in self.df.columns
        
        invalid_scores = self.df[self.df[self.label_col].str.len() != 6]
        if len(invalid_scores) > 0:
            print(f"⚠️ 경고: {len(invalid_scores)}개의 잘못된 BrixiaScore 발견")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        img_name_from_csv = row[self.img_col]
        img_name = img_name_from_csv.replace('.dcm', '.png')
        img_path = os.path.join(self.img_dir, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"❌ 이미지 로드 오류: {img_path}")
            raise
        
        scores_str = row[self.label_col]
        scores_list = [int(c) for c in scores_str]
        labels = torch.tensor(scores_list, dtype=torch.long)  # 🔄 long으로 변경
        
        if self.transform:
            image, labels = self.transform(image, labels)
        else:
            image = TF.to_tensor(image)
            image = TF.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, labels

def create_dataloaders(tr_df, val_df, tt_df, img_dir, 
                       batch_size=32, img_size=224, num_workers=4):
    train_transform = make_transform_with_label(train=True, img_size=img_size)
    val_transform = make_transform_with_label(train=False, img_size=img_size)
    
    tr_ds = BrixiaDataset(tr_df, img_dir, transform=train_transform)
    val_ds = BrixiaDataset(val_df, img_dir, transform=val_transform)
    tt_ds = BrixiaDataset(tt_df, img_dir, transform=val_transform)
    
    tr_loader = DataLoader(
        tr_ds, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available(),
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_ds, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )
    
    tt_loader = DataLoader(
        tt_ds, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=torch.cuda.is_available()
    )
    
    print("✅ DataLoader 준비 완료")
    print(f"   Train: {len(tr_ds)} samples, {len(tr_loader)} batches")
    print(f"   Val:   {len(val_ds)} samples, {len(val_loader)} batches")
    print(f"   Test:  {len(tt_ds)} samples, {len(tt_loader)} batches")
    
    return tr_loader, val_loader, tt_loader

# ============================================================
# Loss Function - Ordinal Classification
# ============================================================
def calculate_class_weights(labels, num_classes=4, method='sqrt_inverse'):
    """클래스 불균형 해결을 위한 가중치 계산"""
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    labels_flat = labels.flatten()
    counts = np.bincount(labels_flat.astype(int), minlength=num_classes)
    
    if method == 'sqrt_inverse':
        weights = 1.0 / (np.sqrt(counts) + 1e-6)
    elif method == 'inverse':
        weights = 1.0 / (counts + 1e-6)
    else:
        total = len(labels_flat)
        weights = total / (num_classes * (counts + 1e-6))
    
    weights = weights / weights.mean()
    return torch.FloatTensor(weights)

def print_class_distribution(labels):
    """클래스 분포 시각화"""
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    
    print("=" * 60)
    print("Class Distribution Analysis")
    print("=" * 60)
    
    region_names = ['A', 'B', 'C', 'D', 'E', 'F']
    for idx, name in enumerate(region_names):
        region_labels = labels[:, idx]
        counts = np.bincount(region_labels.astype(int), minlength=4)
        total = counts.sum()
        
        print(f"\n{name}:")
        for cls in range(4):
            pct = 100 * counts[cls] / total if total > 0 else 0
            bar = '█' * int(pct / 2)
            print(f"  Class {cls}: {counts[cls]:4d} ({pct:5.1f}%) {bar}")
    
    print("=" * 60)

class OrdinalRegressionLoss(nn.Module):
    """
    ✅ Ordinal Regression Loss (순서형 회귀)
    0 < 1 < 2 < 3의 순서를 명시적으로 학습
    """
    def __init__(self, num_classes=4):
        super().__init__()
        self.num_classes = num_classes
        
    def forward(self, logits, target):
        """
        logits: [B*6, num_classes-1] - cumulative logits
        target: [B*6] - class labels (0~3)
        """
        # Cumulative labels 생성
        # Class 0: [0, 0, 0]
        # Class 1: [1, 0, 0]
        # Class 2: [1, 1, 0]
        # Class 3: [1, 1, 1]
        batch_size = target.size(0)
        target_expanded = target.unsqueeze(1)  # [B*6, 1]
        
        # [0, 1, 2, ..., num_classes-2]
        thresholds = torch.arange(self.num_classes - 1).to(target.device)
        thresholds = thresholds.unsqueeze(0).expand(batch_size, -1)  # [B*6, 3]
        
        # target > threshold이면 1, 아니면 0
        cumulative_target = (target_expanded > thresholds).float()  # [B*6, 3]
        
        # Binary cross entropy for each threshold
        loss = F.binary_cross_entropy_with_logits(
            logits, cumulative_target, reduction='none'
        )
        
        return loss.mean()

class AdaptiveOrdinalLoss(nn.Module):
    """
    ✅ Ordinal Loss + Class Weights + Part Weights
    순서를 고려하면서 불균형도 해결
    """
    
    def __init__(self, train_labels, num_classes=4, use_class_weights=True, 
                 part_weights=None, ordinal_weight=0.5):
        super().__init__()
        self.num_classes = num_classes
        self.ordinal_weight = ordinal_weight  # Ordinal loss와 CE loss의 비율
        
        if isinstance(train_labels, torch.Tensor):
            train_labels_np = train_labels.cpu().numpy()
        else:
            train_labels_np = train_labels
        
        print_class_distribution(train_labels_np)
        
        # 클래스 가중치
        if use_class_weights:
            class_weights = calculate_class_weights(train_labels_np, num_classes=num_classes)
            self.register_buffer('class_weights', class_weights)
            print(f"✅ Class weights: {class_weights.numpy()}")
        else:
            self.class_weights = None
        
        # 부위별 가중치
        if part_weights is None:
            self.part_weights = torch.ones(6)
        else:
            self.part_weights = torch.tensor(part_weights, dtype=torch.float32)
        self.register_buffer('part_weights_buf', self.part_weights)
        print(f"✅ Part weights: {self.part_weights.numpy()}")
        print(f"✅ Ordinal weight: {ordinal_weight}")
        
        # Ordinal loss
        self.ordinal_loss = OrdinalRegressionLoss(num_classes)

    def forward(self, pred_dict, target, use_mixup=False):
        """
        pred_dict: {'logits': [B, 6, 4], 'ordinal_logits': [B, 6, 3]}
        target: [B, 6]
        """
        pred_logits = pred_dict['logits']  # [B, 6, 4]
        ordinal_logits = pred_dict['ordinal_logits']  # [B, 6, 3]
        
        B, num_regions, num_classes = pred_logits.shape
        
        # Reshape
        pred_logits_flat = pred_logits.view(B * num_regions, num_classes)
        ordinal_logits_flat = ordinal_logits.view(B * num_regions, num_classes - 1)
        target_flat = target.view(B * num_regions)
        
        # 1. CrossEntropyLoss (기본 분류)
        if self.class_weights is not None and not use_mixup:
            ce_loss = F.cross_entropy(
                pred_logits_flat, 
                target_flat,
                weight=self.class_weights.to(pred_logits.device),
                reduction='none'
            )
        else:
            ce_loss = F.cross_entropy(
                pred_logits_flat, 
                target_flat,
                reduction='none'
            )
        
        # 2. Ordinal Loss (순서 학습)
        ord_loss = self.ordinal_loss(ordinal_logits_flat, target_flat)
        
        # 3. 결합
        total_loss_flat = (1 - self.ordinal_weight) * ce_loss + self.ordinal_weight * ord_loss
        total_loss = total_loss_flat.view(B, num_regions)
        
        # 부위별 가중치 적용
        part_weights = self.part_weights_buf.to(pred_logits.device)
        total_loss = total_loss * part_weights.unsqueeze(0)
        
        part_losses = total_loss.mean(dim=0)
        
        return total_loss.mean(), part_losses

# ============================================================
# Mixup (분류용)
# ============================================================
def mixup_data_classification(x, y, alpha=MIXUP_ALPHA):
    """
    ✅ NEW: 분류용 Mixup
    y는 one-hot으로 변환 후 mixup
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    
    # One-hot encoding
    y_onehot = F.one_hot(y, num_classes=NUM_CLASSES).float()  # [B, 6, 4]
    y_onehot_shuffled = y_onehot[index]
    
    mixed_y = lam * y_onehot + (1 - lam) * y_onehot_shuffled  # [B, 6, 4]
    
    return mixed_x, mixed_y, lam

def mixup_criterion(criterion, pred_dict, y_mixed, lam):
    """Mixup을 위한 손실 계산 (Ordinal Loss 지원)"""
    pred_logits = pred_dict['logits']
    ordinal_logits = pred_dict['ordinal_logits']
    
    B, num_regions, num_classes = pred_logits.shape
    
    # Reshape
    pred_flat = pred_logits.view(B * num_regions, num_classes)
    ordinal_flat = ordinal_logits.view(B * num_regions, num_classes - 1)
    target_flat = y_mixed.view(B * num_regions, num_classes)
    
    # Soft target loss (CE part)
    log_probs = F.log_softmax(pred_flat, dim=1)
    ce_loss = -(target_flat * log_probs).sum(dim=1)
    
    # Ordinal part는 mixup에서 skip (hard label만 사용)
    loss = ce_loss  # Mixup 시에는 ordinal loss 제외
    loss = loss.view(B, num_regions)
    
    # 부위별 가중치 적용
    if hasattr(criterion, 'part_weights_buf'):
        part_weights = criterion.part_weights_buf.to(pred_logits.device)
        loss = loss * part_weights.unsqueeze(0)
    
    part_losses = loss.mean(dim=0)
    return loss.mean(), part_losses

# ============================================================
# Metrics (분류용)
# ============================================================
@torch.no_grad()
def calculate_classification_metrics(pred_dict, labels):
    """
    ✅ 분류 지표 계산
    pred_dict: {'logits': [B, 6, 4], 'ordinal_logits': [B, 6, 3]}
    labels: [B, 6]
    """
    pred_logits = pred_dict['logits']
    
    # 예측 클래스
    preds = pred_logits.argmax(dim=-1)  # [B, 6]
    
    # Exact match accuracy
    exact_acc = (preds == labels).float().mean().item()
    
    # Off-by-1 accuracy (인접 클래스 허용)
    off_by_1 = (torch.abs(preds - labels) <= 1).float().mean().item()
    
    # Per-region accuracy
    region_acc = (preds == labels).float().mean(dim=0)  # [6]
    
    # MAE (참고용)
    mae = torch.abs(preds.float() - labels.float()).mean().item()
    
    return exact_acc, off_by_1, mae, region_acc

# ============================================================
# Model - DenseNet121-MIMIC 기반 (의료 영상 전용)
# ============================================================
class DenseNet121MIMICClassification(nn.Module):
    """
    ✅ DenseNet121-MIMIC 기반 Ordinal Classification 모델
    - MIMIC-CXR 데이터로 사전학습된 의료 영상 전용 모델
    - 각 부위(A~F)마다 4개 클래스(0~3) 예측
    - Self-Attention으로 부위 간 상관관계 학습
    - Ordinal Regression Head 추가 (순서 학습)
    """
    def __init__(self, pretrained=True, drop=0.3, num_regions=6, num_classes=4):
        super().__init__()
        
        # ✅ MIMIC-CXR로 사전학습된 DenseNet121
        if pretrained:
            # self.backbone = xrv.models.DenseNet(weights="densenet121-res224-mimic_ch")
            self.backbone = xrv.models.DenseNet(weights="densenet121-res224-chex")
            print("✅ Loaded DenseNet121 pretrained on CXR!")
        else:
            self.backbone = xrv.models.DenseNet(weights=None)
        
        # DenseNet121의 feature extractor만 사용
        self.features = self.backbone.features
        in_feat = 1024  # DenseNet121의 출력 채널
        
        # Adaptive pooling으로 고정된 크기 출력
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))  # [B, 1024, 7, 7]
        
        # Positional embedding
        self.pos_embed = nn.Parameter(torch.randn(1, 49, in_feat))
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        
        # Region queries
        self.region_queries = nn.Parameter(torch.randn(num_regions, in_feat))
        nn.init.xavier_uniform_(self.region_queries)
        
        # Cross attention (이미지 특징 → 부위별 특징)
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=in_feat,
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )
        
        self.norm1 = nn.LayerNorm(in_feat)
        
        # Self-Attention (부위 간 상관관계 학습)
        self.self_attention = nn.MultiheadAttention(
            embed_dim=in_feat,
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )
        
        self.norm2 = nn.LayerNorm(in_feat)
        self.norm3 = nn.LayerNorm(in_feat)
        
        # FFN
        self.ffn = nn.Sequential(
            nn.Linear(in_feat, in_feat * 2),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(in_feat * 2, in_feat),
            nn.Dropout(drop)
        )
        
        self.norm4 = nn.LayerNorm(in_feat)
        
        # Shared feature extractor
        self.shared_fc = nn.Sequential(
            nn.Linear(in_feat, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(drop)
        )
        
        # ✅ Classification heads (일반 분류)
        self.classification_heads = nn.ModuleList([
            nn.Linear(256, num_classes) for _ in range(num_regions)
        ])
        
        # ✅ Ordinal Regression heads (순서형 회귀)
        self.ordinal_heads = nn.ModuleList([
            nn.Linear(256, num_classes - 1) for _ in range(num_regions)
        ])
    
    def forward(self, x):
        B = x.size(0)
        
        # ✅ DenseNet121 feature extraction
        # torchxrayvision 모델은 grayscale을 기대하므로 RGB의 경우 평균
        if x.size(1) == 3:
            # RGB를 grayscale로 변환 (weighted average)
            x = 0.299 * x[:, 0:1] + 0.587 * x[:, 1:2] + 0.114 * x[:, 2:3]
        
        feat = self.features(x)  # [B, 1024, H, W]
        feat = self.adaptive_pool(feat)  # [B, 1024, 7, 7]
        feat = feat.flatten(2).transpose(1, 2)  # [B, 49, 1024]
        feat = feat + self.pos_embed
        
        # Region queries
        queries = self.region_queries.unsqueeze(0).expand(B, -1, -1)  # [B, 6, 1024]
        
        # Cross attention (이미지 → 부위)
        attn_out, _ = self.cross_attention(
            query=queries,
            key=feat,
            value=feat
        )
        attn_out = self.norm1(attn_out + queries)  # [B, 6, 1024]
        
        # Self-Attention (부위 간 상관관계)
        self_attn_out, attn_weights = self.self_attention(
            query=attn_out,
            key=attn_out,
            value=attn_out
        )
        attn_out = self.norm2(attn_out + self_attn_out)  # [B, 6, 1024]
        
        # FFN
        ffn_out = self.ffn(attn_out)
        attn_out = self.norm3(attn_out + ffn_out)  # [B, 6, 1024]
        
        # Shared feature extraction
        shared_features = self.shared_fc(attn_out)  # [B, 6, 256]
        
        # ✅ 각 부위별 예측 (Classification + Ordinal)
        classification_outputs = []
        ordinal_outputs = []
        
        for i in range(len(self.classification_heads)):
            region_feat = shared_features[:, i, :]  # [B, 256]
            
            # Classification logits
            class_logits = self.classification_heads[i](region_feat)  # [B, 4]
            classification_outputs.append(class_logits)
            
            # Ordinal logits (cumulative)
            ordinal_logits = self.ordinal_heads[i](region_feat)  # [B, 3]
            ordinal_outputs.append(ordinal_logits)
        
        class_out = torch.stack(classification_outputs, dim=1)  # [B, 6, 4]
        ordinal_out = torch.stack(ordinal_outputs, dim=1)  # [B, 6, 3]
        
        return {
            'logits': class_out,
            'ordinal_logits': ordinal_out,
            'attn_weights': attn_weights
        }

# ============================================================
# Training Functions (분류용 수정)
# ============================================================
def train_epoch(model, tr_loader, criterion, optimizer, scaler, device, 
                amp=True, use_mixup=True):
    model.train()
    run_loss = run_acc = run_off1 = run_mae = n = 0
    part_losses_sum = torch.zeros(6)
    
    pbar = tqdm(tr_loader, desc="Train", leave=False)
    
    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)  # [B, 6]
        
        # Mixup 적용
        is_mixup = use_mixup and (random.random() < 0.5)
        
        optimizer.zero_grad(set_to_none=True)
        
        with autocast(enabled=amp):
            if is_mixup:
                imgs_mixed, labels_mixed, lam = mixup_data_classification(imgs, labels)
                pred_dict = model(imgs_mixed)
                loss, part_losses = mixup_criterion(criterion, pred_dict, labels_mixed, lam)
            else:
                pred_dict = model(imgs)
                loss, part_losses = criterion(pred_dict, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Metrics (원본 labels로 계산)
        exact_acc, off_by_1, mae, _ = calculate_classification_metrics(pred_dict, labels)
        
        bs = imgs.size(0)
        run_loss += loss.item() * bs
        run_acc += exact_acc * bs
        run_off1 += off_by_1 * bs
        run_mae += mae * bs
        part_losses_sum += part_losses.cpu() * bs
        n += bs
        
        pbar.set_postfix(
            loss=f"{run_loss/n:.4f}",
            acc=f"{run_acc/n:.4f}",
            mae=f"{run_mae/n:.4f}"
        )
    
    part_losses_avg = part_losses_sum / n
    
    return run_loss/n, run_acc/n, run_off1/n, run_mae/n, part_losses_avg

@torch.no_grad()
def evaluate(model, val_loader, criterion, device, split='val'):
    model.eval()
    run_loss = run_acc = run_off1 = run_mae = n = 0
    part_losses_sum = torch.zeros(6)
    region_acc_sum = torch.zeros(6)
    
    pbar = tqdm(val_loader, desc=f"{split.capitalize()}", leave=False)
    
    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        pred_dict = model(imgs)
        loss, part_losses = criterion(pred_dict, labels)
        
        exact_acc, off_by_1, mae, region_acc = calculate_classification_metrics(pred_dict, labels)
        
        bs = imgs.size(0)
        run_loss += loss.item() * bs
        run_acc += exact_acc * bs
        run_off1 += off_by_1 * bs
        run_mae += mae * bs
        part_losses_sum += part_losses.cpu() * bs
        region_acc_sum += region_acc.cpu() * bs
        n += bs
        
        pbar.set_postfix(
            loss=f"{run_loss/n:.4f}",
            acc=f"{run_acc/n:.4f}"
        )
    
    avg_loss = run_loss/n
    avg_acc = run_acc/n
    avg_off1 = run_off1/n
    avg_mae = run_mae/n
    part_losses_avg = part_losses_sum / n
    region_acc_avg = region_acc_sum / n
    
    print(f"[{split}] loss:{avg_loss:.4f} acc:{avg_acc:.4f} "
          f"off1:{avg_off1:.4f} mae:{avg_mae:.4f}")
    print(f"  Region Acc: {region_acc_avg.numpy().round(3)}")
    
    return avg_loss, avg_acc, avg_off1, avg_mae, part_losses_avg, region_acc_avg

def get_lrs(optimizer):
    return [pg['lr'] for pg in optimizer.param_groups]

# ============================================================
# Main Function
# ============================================================
def main():
    print("\n" + "="*70)
    print("🚀 Brixia COVID-19 Ordinal Classification Training")
    print("   🏥 DenseNet121-MIMIC (의료 영상 전용 사전학습!)")
    print("   💡 80% 목표를 위한 핵심 기술:")
    print("   ✅ MIMIC-CXR 사전학습 모델 (가장 강력!)")
    print("   ✅ Ordinal Loss - 순서형 회귀 (0<1<2<3)")
    print("   ✅ 클래스 불균형 처리 (자동 가중치)")
    print("   ✅ 부위 간 상관관계 학습 (Self-Attention)")
    print("   ✅ Mixed Precision Training (AMP)")
    print("   ✅ Mixup Augmentation")
    print("="*70)
    
    # 데이터 준비
    print("\n📂 Loading data...")
    tr_df, val_df, tt_df = load_and_split_brixia(CSV_PATH)
    
    print("\n📦 Creating DataLoaders...")
    tr_loader, val_loader, tt_loader = create_dataloaders(
        tr_df, val_df, tt_df, img_dir=IMAGE_DIR, 
        batch_size=BATCH_SIZE, img_size=IMG_SIZE, num_workers=4
    )
    
    # Train labels 추출
    train_labels = torch.cat([labels for _, labels in tr_loader], dim=0)
    
    # ========================================
    # 학습 준비
    # ========================================
    print("\n" + "="*70)
    print("📍 Training Setup")
    print("="*70)
    
    # ✅ Ordinal Loss + 클래스 가중치
    criterion = AdaptiveOrdinalLoss(
        train_labels, 
        num_classes=NUM_CLASSES,
        use_class_weights=True,
        part_weights=None,
        ordinal_weight=0.5  # CE:Ordinal = 50:50
    )
    
    # ✅ 의료 영상 전용 사전학습 모델
    model = DenseNet121MIMICClassification(
        pretrained=True, 
        drop=DROP_RATIO,
        num_regions=6,
        num_classes=NUM_CLASSES
    ).to(DEVICE)
    print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, min_lr=1e-6
    )
    scaler = GradScaler(enabled=AMP)
    
    best_acc = 0.0
    best_mae = float('inf')
    patience_counter = 0
    max_patience = 15
    
    # ========================================
    # 학습 루프
    # ========================================
    print("\n" + "="*70)
    print("🏋️ Training Start")
    print("="*70)
    
    for ep in range(1, EPOCHS + 1):
        t0 = time.time()
        
        tr_loss, tr_acc, tr_off1, tr_mae, tr_part_losses = train_epoch(
            model, tr_loader, criterion, optimizer, scaler, DEVICE, AMP
        )
        
        val_loss, val_acc, val_off1, val_mae, val_part_losses, val_region_acc = evaluate(
            model, val_loader, criterion, DEVICE, split='val'
        )
        
        scheduler.step(val_acc)
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            best_mae = val_mae
            patience_counter = 0
            torch.save({
                'epoch': ep,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': best_acc,
                'val_mae': best_mae,
                'val_off1': val_off1,
                'region_acc': val_region_acc,
            }, BEST_PATH)
            print(f"✅ New Best! (Acc={best_acc:.4f}, MAE={best_mae:.4f})")
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= max_patience:
            print(f"\n⏹️ Early stopping at epoch {ep}")
            break
        
        elapsed = time.time() - t0
        print(f"\n[Epoch {ep:03d}/{EPOCHS}]")
        print(f"  Train - loss:{tr_loss:.4f} acc:{tr_acc:.4f} off1:{tr_off1:.4f} mae:{tr_mae:.4f}")
        print(f"  Val   - loss:{val_loss:.4f} acc:{val_acc:.4f} off1:{val_off1:.4f} mae:{val_mae:.4f}")
        print(f"  Part losses (Val): {val_part_losses.numpy().round(3)}")
        print(f"  Region Acc (Val): {val_region_acc.numpy().round(3)}")
        print(f"  LR:{get_lrs(optimizer)[0]:.2e} | {elapsed:.1f}s | Patience:{patience_counter}/{max_patience}")
        print("-" * 70)
    
    # ========================================
    # Test Evaluation
    # ========================================
    print("\n" + "="*70)
    print("🎉 Training Finished!")
    print("="*70)
    
    if len(tt_loader) > 0:
        print("\n📊 Test evaluation with best model...")
        checkpoint = torch.load(BEST_PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        
        tt_loss, tt_acc, tt_off1, tt_mae, tt_part_losses, tt_region_acc = evaluate(
            model, tt_loader, criterion, DEVICE, split='test'
        )
        
        print(f"\n🏆 Test Results:")
        print(f"   Accuracy: {tt_acc:.4f}")
        print(f"   Off-by-1: {tt_off1:.4f}")
        print(f"   MAE: {tt_mae:.4f}")
        print(f"   Region Acc: {tt_region_acc.numpy().round(3)}")
        print(f"   Part losses: {tt_part_losses.numpy().round(3)}")
    
    print(f"\n💾 Best model saved: {BEST_PATH}")
    print(f"📈 Best Validation Accuracy: {best_acc:.4f}")
    print(f"📉 Best Validation MAE: {best_mae:.4f}")
    
    # 성능 향상 제안
    print("\n" + "="*70)
    print("💡 추가 성능 향상을 위한 제안:")
    print("="*70)
    print("1. 🎯 앙상블: 다른 시드로 3~5개 모델 학습 후 투표")
    print("2. 🔬 의료 전용 pretrained model 사용:")
    print("   - CheXpert, MIMIC-CXR 등으로 사전학습된 모델")
    print("3. 📊 데이터 추가:")
    print("   - 외부 COVID-19 데이터셋 활용")
    print("   - Pseudo-labeling으로 unlabeled 데이터 활용")
    print("4. 🎨 고급 증강:")
    print("   - CutMix, AugMix, RandAugment")
    print("   - Test-Time Augmentation (TTA)")
    print("5. 🧪 하이퍼파라미터 튜닝:")
    print("   - ordinal_weight 조정 (0.3~0.7)")
    print("   - Learning rate, batch size 실험")
    print("="*70)
    
    print("\n✅ 이제 gradcam_inference.py를 실행하여 결과를 시각화하세요!")
    print("="*70)

if __name__ == "__main__":
    main()


🚀 Brixia COVID-19 Ordinal Classification Training
   🏥 DenseNet121-MIMIC (의료 영상 전용 사전학습!)
   💡 80% 목표를 위한 핵심 기술:
   ✅ MIMIC-CXR 사전학습 모델 (가장 강력!)
   ✅ Ordinal Loss - 순서형 회귀 (0<1<2<3)
   ✅ 클래스 불균형 처리 (자동 가중치)
   ✅ 부위 간 상관관계 학습 (Self-Attention)
   ✅ Mixed Precision Training (AMP)
   ✅ Mixup Augmentation

📂 Loading data...
유효한 데이터: 4695개
Train: 3637, Val: 912, Test: 146
Train - Mean: 8.31, Std: 4.26
Val - Mean: 8.35, Std: 4.15
Test - Mean: 7.78, Std: 4.20

📦 Creating DataLoaders...
✅ DataLoader 준비 완료
   Train: 3637 samples, 113 batches
   Val:   912 samples, 29 batches
   Test:  146 samples, 5 batches

📍 Training Setup
Class Distribution Analysis

A:
  Class 0: 1810 ( 50.1%) █████████████████████████
  Class 1: 1126 ( 31.1%) ███████████████
  Class 2:  446 ( 12.3%) ██████
  Class 3:  234 (  6.5%) ███

B:
  Class 0:  721 ( 19.9%) █████████
  Class 1:  949 ( 26.2%) █████████████
  Class 2: 1171 ( 32.4%) ████████████████
  Class 3:  775 ( 21.4%) ██████████

C:
  Class 0:  406 ( 11.2%) █████


[val] loss:0.8672 acc:0.4346 off1:0.8299 mae:0.7577
  Region Acc: [0.471 0.373 0.404 0.578 0.374 0.408]
✅ New Best! (Acc=0.4346, MAE=0.7577)

[Epoch 001/100]
  Train - loss:1.0916 acc:0.3780 off1:0.8085 mae:0.8496
  Val   - loss:0.8672 acc:0.4346 off1:0.8299 mae:0.7577
  Part losses (Val): [0.847 0.924 0.884 0.753 0.911 0.884]
  Region Acc (Val): [0.471 0.373 0.404 0.578 0.374 0.408]
  LR:1.00e-04 | 20.3s | Patience:0/15
----------------------------------------------------------------------


[val] loss:0.7997 acc:0.4713 off1:0.8628 mae:0.6838
  Region Acc: [0.525 0.407 0.416 0.607 0.414 0.458]
✅ New Best! (Acc=0.4713, MAE=0.6838)

[Epoch 002/100]
  Train - loss:1.0302 acc:0.4215 off1:0.8358 mae:0.7681
  Val   - loss:0.7997 acc:0.4713 off1:0.8628 mae:0.6838
  Part losses (Val): [0.774 0.837 0.839 0.695 0.829 0.824]
  Region Acc (Val): [0.525 0.407 0.416 0.607 0.414 0.458]
  LR:1.00e-04 | 18.3s | Patience:0/15
----------------------------------------------------------------------


[val] loss:0.7686 acc:0.4839 off1:0.8792 mae:0.6488
  Region Acc: [0.532 0.445 0.438 0.614 0.419 0.456]
✅ New Best! (Acc=0.4839, MAE=0.6488)

[Epoch 003/100]
  Train - loss:0.9762 acc:0.4330 off1:0.8454 mae:0.7468
  Val   - loss:0.7686 acc:0.4839 off1:0.8792 mae:0.6488
  Part losses (Val): [0.749 0.791 0.806 0.675 0.793 0.798]
  Region Acc (Val): [0.532 0.445 0.438 0.614 0.419 0.456]
  LR:1.00e-04 | 17.7s | Patience:0/15
----------------------------------------------------------------------


[val] loss:0.7622 acc:0.4978 off1:0.8867 mae:0.6232
  Region Acc: [0.544 0.467 0.452 0.612 0.452 0.461]
✅ New Best! (Acc=0.4978, MAE=0.6232)

[Epoch 004/100]
  Train - loss:0.9641 acc:0.4335 off1:0.8455 mae:0.7491
  Val   - loss:0.7622 acc:0.4978 off1:0.8867 mae:0.6232
  Part losses (Val): [0.739 0.775 0.798 0.682 0.787 0.793]
  Region Acc (Val): [0.544 0.467 0.452 0.612 0.452 0.461]
  LR:1.00e-04 | 19.1s | Patience:0/15
----------------------------------------------------------------------


...

[val] loss:0.6524 acc:0.5671 off1:0.9432 mae:0.4942
  Region Acc: [0.6   0.554 0.55  0.649 0.55  0.499]

[Epoch 075/100]
  Train - loss:0.7535 acc:0.5369 off1:0.9142 mae:0.5637
  Val   - loss:0.6524 acc:0.5671 off1:0.9432 mae:0.4942
  Part losses (Val): [0.639 0.659 0.669 0.561 0.671 0.716]
  Region Acc (Val): [0.6   0.554 0.55  0.649 0.55  0.499]
  LR:1.00e-06 | 19.0s | Patience:12/15
----------------------------------------------------------------------


[val] loss:0.6509 acc:0.5660 off1:0.9433 mae:0.4947
  Region Acc: [0.591 0.545 0.554 0.652 0.555 0.499]

[Epoch 076/100]
  Train - loss:0.8149 acc:0.5184 off1:0.8945 mae:0.6076
  Val   - loss:0.6509 acc:0.5660 off1:0.9433 mae:0.4947
  Part losses (Val): [0.636 0.658 0.669 0.561 0.668 0.714]
  Region Acc (Val): [0.591 0.545 0.554 0.652 0.555 0.499]
  LR:1.00e-06 | 15.9s | Patience:13/15
----------------------------------------------------------------------


[val] loss:0.6527 acc:0.5665 off1:0.9439 mae:0.4940
  Region Acc: [0.6   0.55  0.547 0.65  0.552 0.5  ]

[Epoch 077/100]
  Train - loss:0.7989 acc:0.5044 off1:0.8874 mae:0.6282
  Val   - loss:0.6527 acc:0.5665 off1:0.9439 mae:0.4940
  Part losses (Val): [0.638 0.658 0.669 0.561 0.674 0.717]
  Region Acc (Val): [0.6   0.55  0.547 0.65  0.552 0.5  ]
  LR:1.00e-06 | 18.3s | Patience:14/15
----------------------------------------------------------------------


[val] loss:0.6558 acc:0.5627 off1:0.9421 mae:0.4996
  Region Acc: [0.594 0.545 0.532 0.65  0.549 0.505]

⏹️ Early stopping at epoch 78

🎉 Training Finished!

📊 Test evaluation with best model...


[test] loss:0.6052 acc:0.5765 off1:0.9623 mae:0.4646
  Region Acc: [0.568 0.582 0.534 0.692 0.603 0.479]

🏆 Test Results:
   Accuracy: 0.5765
   Off-by-1: 0.9623
   MAE: 0.4646
   Region Acc: [0.568 0.582 0.534 0.692 0.603 0.479]
   Part losses: [0.593 0.63  0.62  0.507 0.599 0.682]

💾 Best model saved: runs_severity_classification/best_densenet121_mimic_classification.pth
📈 Best Validation Accuracy: 0.5707
📉 Best Validation MAE: 0.4918

💡 추가 성능 향상을 위한 제안:
1. 🎯 앙상블: 다른 시드로 3~5개 모델 학습 후 투표
2. 🔬 의료 전용 pretrained model 사용:
   - CheXpert, MIMIC-CXR 등으로 사전학습된 모델
3. 📊 데이터 추가:
   - 외부 COVID-19 데이터셋 활용
   - Pseudo-labeling으로 unlabeled 데이터 활용
4. 🎨 고급 증강:
   - CutMix, AugMix, RandAugment
   - Test-Time Augmentation (TTA)
5. 🧪 하이퍼파라미터 튜닝:
   - ordinal_weight 조정 (0.3~0.7)
   - Learning rate, batch size 실험

✅ 이제 gradcam_inference.py를 실행하여 결과를 시각화하세요!


In [8]:
pip install torchxrayvision

Looking in indexes: https://mirror.kakao.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 2.7 MB/s  0:00:10m0:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 2.1 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [torchxrayvision] [scikit-image]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
